In [1]:
import pandas as pd
!pip install geopy
from geopy.geocoders import Nominatim
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install folium
import folium


chester_county_pa = {'Borough': ['Atglen', 'Avondale', 'Downingtown', 'Elverson', 'Honey Brook', 'Kennett Square', 'Malvern', 'Modena', 'Oxford', 'Parkesburg', 'Phoenixville', 'South Coatesville', 'Spring City', 'West Chester', 'West Grove'],
                    'Latitude': ['39.9493', '39.8234', '40.0065', '40.1568', '40.0943', '39.8468', '40.0362', '39.9611', '39.7854', '39.9594', '40.1304', '39.9743', '40.1768', '39.9607', '39.8221'],
                     'Longitude': ['-75.9736', '-75.7833', '-75.7033', '-75.8327', '-75.9113', '-75.7116', '-75.5138', '-75.7996', '-75.9788', '-75.9168', '-75.5149', '-75.8199', '-75.5477', '-75.6055', '-75.8274'],
                    }
df = pd.DataFrame(chester_county_pa, columns=['Borough', 'Latitude', 'Longitude'])
df

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


,Borough,Latitude,Longitude
0,Atglen,39.9493,-75.9736
1,Avondale,39.8234,-75.7833
2,Downingtown,40.0065,-75.7033
3,Elverson,40.1568,-75.8327
4,Honey Brook,40.0943,-75.9113
5,Kennett Square,39.8468,-75.7116
6,Malvern,40.0362,-75.5138
7,Modena,39.9611,-75.7996
8,Oxford,39.7854,-75.9788
9,Parkesburg,39.9594,-75.9168


In [2]:
address = '100 Vanguard Blvd, PA'

geolocator = Nominatim(user_agent="Malvern")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Malvern are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Malvern are 40.05212105, -75.50690726044587.


In [3]:
map1 = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, borough, in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map1)  
    
map1

dfm = {'Borough': ['Malvern', 'Malvern', 'Malvern', 'Malvern', 'Malvern', 'Malvern', 'Malvern', 'Malvern', 'Malvern', 'Malvern', 'Malvern'],
       'Neighborhood':['Frazer', 'General Warren Village', 'Planebrook', 'Charlestown', 'Devault', 'Cedar Hollow', 'Howellville', 'Immaculata U', 'Newtown Square', 'Willistown', 'Whitehorse'],
      'Latitude': ['40.0323', '40.0418','40.0396','40.0852', '40.0736', '40.0726','40.0596','40.0280', '39.9869', '39.9916', '40.00348'],
      'Longitude': ['-75.5560', '-75.5233','-75.5738','-75.5568', '-75.5355', '-75.5227','-75.4696','-75.5707', '-75.4007', '-75.4927','-75.9938'],
      }
dfm = pd.DataFrame(dfm, columns=['Borough', 'Neighborhood', 'Latitude', 'Longitude'])
dfm

In [4]:
CLIENT_ID = 'GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO' # your Foursquare ID
CLIENT_SECRET = 'WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500

In [5]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Atglen are 39.9493, -75.9736.


In [6]:
LIMIT = 500
radius = 1600
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&v=20180604&ll=39.9493,-75.9736&radius=1600&limit=500'

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=1600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
         # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Atglen
Avondale
Downingtown
Elverson
Honey Brook
Kennett Square
Malvern
Modena
Oxford
Parkesburg
Phoenixville
South Coatesville
Spring City
West Chester
West Grove


In [9]:
venues.head(50)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Atglen,39.9493,-75.9736,Reaching Higher LLC,39.956712,-75.980964,Construction & Landscaping
1,Atglen,39.9493,-75.9736,Schiffer Book Farm,39.948898,-75.957474,Bookstore
2,Avondale,39.8234,-75.7833,Va La Vineyards,39.817111,-75.772314,Vineyard
3,Avondale,39.8234,-75.7833,Glen Willow Orchards,39.835500,-75.785355,Farmers Market
4,Avondale,39.8234,-75.7833,Domino's Pizza,39.823428,-75.781126,Pizza Place
5,Avondale,39.8234,-75.7833,Dunkin',39.823372,-75.780938,Donut Shop
6,Avondale,39.8234,-75.7833,Walgreens,39.834767,-75.793396,Pharmacy
7,Avondale,39.8234,-75.7833,Outfield Landscaping,39.813218,-75.775551,Construction & Landscaping
8,Avondale,39.8234,-75.7833,Jd Mushrooms,39.812199,-75.776009,Food
9,Avondale,39.8234,-75.7833,Walgreens,39.817253,-75.768410,Pharmacy


In [10]:
print(venues.shape)
venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

(402, 7)
There are 121 unique categories.


In [11]:
venue_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
venue_onehot.insert(loc=0, column='Neighborhood', value=venues['Neighborhood'])
venue_onehot.shape

(402, 122)

In [12]:
venues_grouped = venue_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,...,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Travel Agency,Vegetarian / Vegan Restaurant,Video Store,Vineyard,Whisky Bar
0,Atglen,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.00
1,Avondale,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.111111,0.00
2,Downingtown,0.044444,0.00,0.000000,0.00,0.000000,0.0,0.00,0.044444,0.000000,...,0.000000,0.000000,0.000000,0.022222,0.000,0.00,0.00,0.000000,0.000000,0.00
3,Elverson,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.200,0.00,0.00,0.000000,0.000000,0.00
4,Honey Brook,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,...,0.000000,0.000000,0.076923,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.00
5,Kennett Square,0.018868,0.00,0.000000,0.00,0.000000,0.0,0.00,0.037736,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.00
6,Malvern,0.100000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.025,0.00,0.00,0.000000,0.000000,0.00
7,Modena,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,...,0.333333,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.00
8,Oxford,0.000000,0.00,0.000000,0.00,0.045455,0.0,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.00
9,Parkesburg,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.111111,0.000000,0.00


In [13]:
num_top_venues = 5

for hood in venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = venues_grouped[venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Atglen----
                        venue  freq
0  Construction & Landscaping   0.5
1                   Bookstore   0.5
2         American Restaurant   0.0
3                      Lounge   0.0
4                    Pharmacy   0.0


----Avondale----
                        venue  freq
0                    Pharmacy  0.22
1  Construction & Landscaping  0.11
2                 Pizza Place  0.11
3                  Donut Shop  0.11
4              Farmers Market  0.11


----Downingtown----
                  venue  freq
0   American Restaurant  0.04
1                Bakery  0.04
2           Gas Station  0.04
3               Brewery  0.04
4  Gym / Fitness Center  0.04


----Elverson----
                        venue  freq
0  Construction & Landscaping   0.2
1                        Park   0.2
2               Train Station   0.2
3          Italian Restaurant   0.2
4           Elementary School   0.2


----Honey Brook----
           venue  freq
0       Pharmacy  0.15
1    Gas Station  0.15
2     

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atglen,Construction & Landscaping,Bookstore,Whisky Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
1,Avondale,Pharmacy,Construction & Landscaping,Farmers Market,Food,Donut Shop,Pizza Place,Grocery Store,Vineyard,College Auditorium,Concert Hall
2,Downingtown,American Restaurant,Brewery,Italian Restaurant,Gym / Fitness Center,Gas Station,Convenience Store,Coffee Shop,Pizza Place,Bakery,Café
3,Elverson,Park,Italian Restaurant,Construction & Landscaping,Train Station,Elementary School,College Auditorium,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio
4,Honey Brook,Gas Station,Pharmacy,Restaurant,Italian Restaurant,Grocery Store,Chinese Restaurant,Toy / Game Store,Pizza Place,Brewery,Elementary School


In [16]:
from sklearn.cluster import KMeans

kclusters = 5

chester_cluster = venues_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chester_cluster)

kmeans.labels_[0:10]

array([0, 1, 1, 3, 1, 1, 1, 2, 1, 1], dtype=int32)

In [17]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atglen,Construction & Landscaping,Bookstore,Whisky Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
1,Avondale,Pharmacy,Construction & Landscaping,Farmers Market,Food,Donut Shop,Pizza Place,Grocery Store,Vineyard,College Auditorium,Concert Hall
2,Downingtown,American Restaurant,Brewery,Italian Restaurant,Gym / Fitness Center,Gas Station,Convenience Store,Coffee Shop,Pizza Place,Bakery,Café
3,Elverson,Park,Italian Restaurant,Construction & Landscaping,Train Station,Elementary School,College Auditorium,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio
4,Honey Brook,Gas Station,Pharmacy,Restaurant,Italian Restaurant,Grocery Store,Chinese Restaurant,Toy / Game Store,Pizza Place,Brewery,Elementary School
5,Kennett Square,Italian Restaurant,Pizza Place,Gym / Fitness Center,Breakfast Spot,Mexican Restaurant,Baseball Field,Gastropub,Discount Store,Bakery,Paper / Office Supplies Store
6,Malvern,American Restaurant,Sandwich Place,Pizza Place,Big Box Store,Hotel,Italian Restaurant,Coffee Shop,Playground,Photography Studio,Pharmacy
7,Modena,Farm,Park,Thai Restaurant,Elementary School,College Auditorium,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio
8,Oxford,Fast Food Restaurant,Discount Store,Big Box Store,Liquor Store,Pet Store,Pharmacy,Dance Studio,Pub,Business Service,Italian Restaurant
9,Parkesburg,Pharmacy,Gym / Fitness Center,Discount Store,Light Rail Station,Italian Restaurant,Ice Cream Shop,Gas Station,Fruit & Vegetable Store,Video Store,Football Stadium


In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chester_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
chester_merged = chester_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

chester_merged # check the last columns!

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atglen,39.9493,-75.9736,0,Construction & Landscaping,Bookstore,Whisky Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
1,Avondale,39.8234,-75.7833,1,Pharmacy,Construction & Landscaping,Farmers Market,Food,Donut Shop,Pizza Place,Grocery Store,Vineyard,College Auditorium,Concert Hall
2,Downingtown,40.0065,-75.7033,1,American Restaurant,Brewery,Italian Restaurant,Gym / Fitness Center,Gas Station,Convenience Store,Coffee Shop,Pizza Place,Bakery,Café
3,Elverson,40.1568,-75.8327,3,Park,Italian Restaurant,Construction & Landscaping,Train Station,Elementary School,College Auditorium,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio
4,Honey Brook,40.0943,-75.9113,1,Gas Station,Pharmacy,Restaurant,Italian Restaurant,Grocery Store,Chinese Restaurant,Toy / Game Store,Pizza Place,Brewery,Elementary School
5,Kennett Square,39.8468,-75.7116,1,Italian Restaurant,Pizza Place,Gym / Fitness Center,Breakfast Spot,Mexican Restaurant,Baseball Field,Gastropub,Discount Store,Bakery,Paper / Office Supplies Store
6,Malvern,40.0362,-75.5138,1,American Restaurant,Sandwich Place,Pizza Place,Big Box Store,Hotel,Italian Restaurant,Coffee Shop,Playground,Photography Studio,Pharmacy
7,Modena,39.9611,-75.7996,2,Farm,Park,Thai Restaurant,Elementary School,College Auditorium,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio
8,Oxford,39.7854,-75.9788,1,Fast Food Restaurant,Discount Store,Big Box Store,Liquor Store,Pet Store,Pharmacy,Dance Studio,Pub,Business Service,Italian Restaurant
9,Parkesburg,39.9594,-75.9168,1,Pharmacy,Gym / Fitness Center,Discount Store,Light Rail Station,Italian Restaurant,Ice Cream Shop,Gas Station,Fruit & Vegetable Store,Video Store,Football Stadium


In [19]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chester_merged['Latitude'], chester_merged['Longitude'], chester_merged['Borough'], chester_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [20]:
chester_merged.loc[chester_merged['Cluster Labels'] == 0, chester_merged.columns[[1] + list(range(0, chester_merged.shape[1]))]]

,Latitude,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,39.9493,Atglen,39.9493,-75.9736,0,Construction & Landscaping,Bookstore,Whisky Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop


In [21]:
chester_merged.loc[chester_merged['Cluster Labels'] == 1, chester_merged.columns[[1] + list(range(0, chester_merged.shape[1]))]]

,Latitude,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,39.8234,Avondale,39.8234,-75.7833,1,Pharmacy,Construction & Landscaping,Farmers Market,Food,Donut Shop,Pizza Place,Grocery Store,Vineyard,College Auditorium,Concert Hall
2,40.0065,Downingtown,40.0065,-75.7033,1,American Restaurant,Brewery,Italian Restaurant,Gym / Fitness Center,Gas Station,Convenience Store,Coffee Shop,Pizza Place,Bakery,Café
4,40.0943,Honey Brook,40.0943,-75.9113,1,Gas Station,Pharmacy,Restaurant,Italian Restaurant,Grocery Store,Chinese Restaurant,Toy / Game Store,Pizza Place,Brewery,Elementary School
5,39.8468,Kennett Square,39.8468,-75.7116,1,Italian Restaurant,Pizza Place,Gym / Fitness Center,Breakfast Spot,Mexican Restaurant,Baseball Field,Gastropub,Discount Store,Bakery,Paper / Office Supplies Store
6,40.0362,Malvern,40.0362,-75.5138,1,American Restaurant,Sandwich Place,Pizza Place,Big Box Store,Hotel,Italian Restaurant,Coffee Shop,Playground,Photography Studio,Pharmacy
8,39.7854,Oxford,39.7854,-75.9788,1,Fast Food Restaurant,Discount Store,Big Box Store,Liquor Store,Pet Store,Pharmacy,Dance Studio,Pub,Business Service,Italian Restaurant
9,39.9594,Parkesburg,39.9594,-75.9168,1,Pharmacy,Gym / Fitness Center,Discount Store,Light Rail Station,Italian Restaurant,Ice Cream Shop,Gas Station,Fruit & Vegetable Store,Video Store,Football Stadium
10,40.1304,Phoenixville,40.1304,-75.5149,1,American Restaurant,Brewery,Coffee Shop,Pizza Place,Bank,Sandwich Place,Italian Restaurant,Liquor Store,Chinese Restaurant,Café
11,39.9743,South Coatesville,39.9743,-75.8199,1,Pharmacy,Trail,Clothing Store,Farmers Market,Fast Food Restaurant,Convenience Store,Pizza Place,Gas Station,Diner,College Auditorium
12,40.1768,Spring City,40.1768,-75.5477,1,Pizza Place,Baseball Field,Italian Restaurant,Breakfast Spot,Boutique,Café,Pub,Farm,Home Service,Diner


In [22]:
chester_merged.loc[chester_merged['Cluster Labels'] == 2, chester_merged.columns[[1] + list(range(0, chester_merged.shape[1]))]]

,Latitude,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,39.9611,Modena,39.9611,-75.7996,2,Farm,Park,Thai Restaurant,Elementary School,College Auditorium,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio


In [23]:
chester_merged.loc[chester_merged['Cluster Labels'] == 3, chester_merged.columns[[1] + list(range(0, chester_merged.shape[1]))]]

,Latitude,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,40.1568,Elverson,40.1568,-75.8327,3,Park,Italian Restaurant,Construction & Landscaping,Train Station,Elementary School,College Auditorium,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio


In [24]:
chester_merged.loc[chester_merged['Cluster Labels'] == 4, chester_merged.columns[[1] + list(range(0, chester_merged.shape[1]))]]

,Latitude,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,39.8221,West Grove,39.8221,-75.8274,4,Park,Baseball Field,American Restaurant,Automotive Shop,Deli / Bodega,Bank,Pizza Place,Furniture / Home Store,Construction & Landscaping,Convenience Store


# Personal Search: Melvern

In [25]:
address = '100 Vanguard Malvern, PA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.0521503 -75.505871


# Church

In [26]:

search_query = 'church'
radius = 16093
print('done')

done


In [27]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=church&radius=16093&limit=500'

In [28]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3c96103eb707d00b9a36'},
 'response': {'venues': [{'id': '4e2ef003e4cdb5a70d631cec',
    'name': 'The Church of Saint Monica',
    'location': {'address': '63 Main Ave',
     'crossStreet': 'at First Ave',
     'lat': 40.04608705725427,
     'lng': -75.44035900501834,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.04608705725427,
       'lng': -75.44035900501834}],
     'distance': 5623,
     'postalCode': '19312',
     'cc': 'US',
     'city': 'Berwyn',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['63 Main Ave (at First Ave)',
      'Berwyn, PA 19312',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d132941735',
      'name': 'Church',
      'pluralName': 'Churches',
      'shortName': 'Church',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/religious_church_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595620732',
    'has

In [29]:
from pandas.io.json import json_normalize
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4e2ef003e4cdb5a70d631cec,63 Main Ave,US,Berwyn,United States,at First Ave,5623,"[63 Main Ave (at First Ave), Berwyn, PA 19312,...","[{'label': 'display', 'lat': 40.04608705725427...",40.046087,-75.440359,19312,PA,The Church of Saint Monica,v-1595620732,NaN
1,"[{'id': '4bf58dd8d48988d1a8941735', 'name': 'G...",False,4c3de78f15cb1b8d198f7cdc,1001 E Lincoln Hwy,US,Exton,United States,NaN,7889,"[1001 E Lincoln Hwy, Exton, PA 19341, United S...","[{'label': 'display', 'lat': 40.03273681214128...",40.032737,-75.594901,19341,PA,Church Farm School,v-1595620732,NaN
2,[],False,5577e96b498e7a29340213b3,NaN,US,Paoli,United States,NaN,1722,"[Paoli, PA, United States]","[{'label': 'display', 'lat': 40.0415981, 'lng'...",40.041598,-75.491087,NaN,PA,Church of the Good Samaritan,v-1595620732,NaN
3,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4b7c9e6ef964a520189e2fe3,Lancaster Ave,US,Paoli,United States,NaN,1807,"[Lancaster Ave, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04091559619378...",40.040916,-75.490552,19301,PA,Church of the Good Samaritan,v-1595620732,NaN
4,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4ceb25c1baa6a1cde4fc3c6c,50 W Walker Rd,US,Wayne,United States,NaN,6026,"[50 W Walker Rd, Wayne, PA 19087, United States]","[{'label': 'display', 'lat': 40.07388665956120...",40.073887,-75.441086,19087,PA,St. Isaac Jogues Catholic Church,v-1595620732,NaN
5,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4d15785d6d103704171d2abd,199 St. Marys St.,US,Phoenixville,United States,Dayton St.,9852,"[199 St. Marys St. (Dayton St.), Phoenixville,...","[{'label': 'display', 'lat': 40.14046, 'lng': ...",40.140460,-75.513549,19460,PA,St Mary Of The Assumption Church,v-1595620732,NaN
6,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,539e0497498e71ce2a63cea3,NaN,US,Phoenixville,United States,NaN,1459,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.03923316394582...",40.039233,-75.508792,NaN,PA,"First Church Of Christ, Scientist",v-1595620732,NaN
7,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4e19b6f088777e96477c526e,NaN,US,Phoenixville,United States,NaN,7620,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.12007149849076...",40.120071,-75.517039,NaN,PA,Christian Science Church,v-1595620732,NaN
8,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4ef6209e0aafbcb31b6551b9,Landowner Road,US,Malvern,United States,NaN,1571,"[Landowner Road, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.038544, 'lng':...",40.038544,-75.510767,19355,PA,The Methodist Church,v-1595620732,NaN
9,"[{'id': '4bf58dd8d48988d1f9931735', 'name': 'R...",False,4ed4de8c0e61888eaad73bd6,N Church St.,US,Spring City,United States,at Yost Ave.,14700,"[N Church St. (at Yost Ave.), Spring City, PA ...","[{'label': 'display', 'lat': 40.179916, 'lng':...",40.179916,-75.549541,19475,PA,Church Street & Yost Avenue,v-1595620732,NaN


In [30]:

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Church of Saint Monica,Church,63 Main Ave,US,Berwyn,United States,at First Ave,5623,"[63 Main Ave (at First Ave), Berwyn, PA 19312,...","[{'label': 'display', 'lat': 40.04608705725427...",40.046087,-75.440359,19312,PA,4e2ef003e4cdb5a70d631cec
1,Church Farm School,General College & University,1001 E Lincoln Hwy,US,Exton,United States,NaN,7889,"[1001 E Lincoln Hwy, Exton, PA 19341, United S...","[{'label': 'display', 'lat': 40.03273681214128...",40.032737,-75.594901,19341,PA,4c3de78f15cb1b8d198f7cdc
2,Church of the Good Samaritan,None,NaN,US,Paoli,United States,NaN,1722,"[Paoli, PA, United States]","[{'label': 'display', 'lat': 40.0415981, 'lng'...",40.041598,-75.491087,NaN,PA,5577e96b498e7a29340213b3
3,Church of the Good Samaritan,Church,Lancaster Ave,US,Paoli,United States,NaN,1807,"[Lancaster Ave, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04091559619378...",40.040916,-75.490552,19301,PA,4b7c9e6ef964a520189e2fe3
4,St. Isaac Jogues Catholic Church,Church,50 W Walker Rd,US,Wayne,United States,NaN,6026,"[50 W Walker Rd, Wayne, PA 19087, United States]","[{'label': 'display', 'lat': 40.07388665956120...",40.073887,-75.441086,19087,PA,4ceb25c1baa6a1cde4fc3c6c
5,St Mary Of The Assumption Church,Church,199 St. Marys St.,US,Phoenixville,United States,Dayton St.,9852,"[199 St. Marys St. (Dayton St.), Phoenixville,...","[{'label': 'display', 'lat': 40.14046, 'lng': ...",40.140460,-75.513549,19460,PA,4d15785d6d103704171d2abd
6,"First Church Of Christ, Scientist",Church,NaN,US,Phoenixville,United States,NaN,1459,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.03923316394582...",40.039233,-75.508792,NaN,PA,539e0497498e71ce2a63cea3
7,Christian Science Church,Church,NaN,US,Phoenixville,United States,NaN,7620,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.12007149849076...",40.120071,-75.517039,NaN,PA,4e19b6f088777e96477c526e
8,The Methodist Church,Church,Landowner Road,US,Malvern,United States,NaN,1571,"[Landowner Road, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.038544, 'lng':...",40.038544,-75.510767,19355,PA,4ef6209e0aafbcb31b6551b9
9,Church Street & Yost Avenue,Road,N Church St.,US,Spring City,United States,at Yost Ave.,14700,"[N Church St. (at Yost Ave.), Spring City, PA ...","[{'label': 'display', 'lat': 40.179916, 'lng':...",40.179916,-75.549541,19475,PA,4ed4de8c0e61888eaad73bd6


In [31]:
dataframe_filtered.shape

(50, 15)

In [32]:
df_church_pa = dataframe_filtered[dataframe_filtered.categories == 'Church']
df_church_pa.reset_index(inplace=True, drop=True)
df_church_pa

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Church of Saint Monica,Church,63 Main Ave,US,Berwyn,United States,at First Ave,5623,"[63 Main Ave (at First Ave), Berwyn, PA 19312,...","[{'label': 'display', 'lat': 40.04608705725427...",40.046087,-75.440359,19312,PA,4e2ef003e4cdb5a70d631cec
1,Church of the Good Samaritan,Church,Lancaster Ave,US,Paoli,United States,NaN,1807,"[Lancaster Ave, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04091559619378...",40.040916,-75.490552,19301,PA,4b7c9e6ef964a520189e2fe3
2,St. Isaac Jogues Catholic Church,Church,50 W Walker Rd,US,Wayne,United States,NaN,6026,"[50 W Walker Rd, Wayne, PA 19087, United States]","[{'label': 'display', 'lat': 40.07388665956120...",40.073887,-75.441086,19087,PA,4ceb25c1baa6a1cde4fc3c6c
3,St Mary Of The Assumption Church,Church,199 St. Marys St.,US,Phoenixville,United States,Dayton St.,9852,"[199 St. Marys St. (Dayton St.), Phoenixville,...","[{'label': 'display', 'lat': 40.14046, 'lng': ...",40.140460,-75.513549,19460,PA,4d15785d6d103704171d2abd
4,"First Church Of Christ, Scientist",Church,NaN,US,Phoenixville,United States,NaN,1459,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.03923316394582...",40.039233,-75.508792,NaN,PA,539e0497498e71ce2a63cea3
5,Christian Science Church,Church,NaN,US,Phoenixville,United States,NaN,7620,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.12007149849076...",40.120071,-75.517039,NaN,PA,4e19b6f088777e96477c526e
6,The Methodist Church,Church,Landowner Road,US,Malvern,United States,NaN,1571,"[Landowner Road, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.038544, 'lng':...",40.038544,-75.510767,19355,PA,4ef6209e0aafbcb31b6551b9
7,St. David's Episcopal Church,Church,762 Valley Forge Rd,US,Wayne,United States,btwn Church Rd & Maplewood Rd,9127,[762 Valley Forge Rd (btwn Church Rd & Maplewo...,"[{'label': 'display', 'lat': 40.02697026100816...",40.026970,-75.403951,19087,PA,4e19d50dd4c062b044f53f58
8,"Saint Luke Lutheran Church - Devon, PA",Church,203 N Valley Forge Rd,US,Devon,United States,Conestoga Rd.,7212,"[203 N Valley Forge Rd (Conestoga Rd.), Devon,...","[{'label': 'display', 'lat': 40.05066761991265...",40.050668,-75.421251,19333,PA,4ec0362b30f82a2e13521d83
9,Great Valley Presbyterian Church,Church,2025 Swedesford Rd,US,Malvern,United States,NaN,1380,"[2025 Swedesford Rd, Malvern, PA 19355, United...","[{'label': 'display', 'lat': 40.06264683752855...",40.062647,-75.497251,19355,PA,4b9d4366f964a520319e36e3


In [33]:
df_church_pa.shape

(41, 15)

# Aldi

In [34]:
search_query = 'Aldi'
radius = 16093
print('done')

done


In [35]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Aldi&radius=16093&limit=500'

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3ce8fcc0f268b53bd271'},
 'response': {'venues': [{'id': '5d0681096612700023aa5258',
    'name': 'ALDI',
    'location': {'address': '11 Matthews Rd',
     'lat': 40.05055667,
     'lng': -75.52334167,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.05055667,
       'lng': -75.52334167},
      {'label': 'entrance', 'lat': 40.050426, 'lng': -75.523471}],
     'distance': 1499,
     'postalCode': '19355',
     'cc': 'US',
     'city': 'Malvern',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['11 Matthews Rd',
      'Malvern, PA 19355',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595620733',
    'hasPerk': Fal

In [37]:
venues = results['response']['venues']
dataframe= json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,5d0681096612700023aa5258,11 Matthews Rd,US,Malvern,United States,NaN,1499,"[11 Matthews Rd, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.05055667, 'lng...",40.050557,-75.523342,19355,PA,ALDI,v-1595620733
1,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4d87bc1f7d4c5481a1723271,550 S Trooper Rd,US,Norristown,United States,at Egypt Rd,12537,"[550 S Trooper Rd (at Egypt Rd), Norristown, P...","[{'label': 'display', 'lat': 40.1293825, 'lng'...",40.129382,-75.398713,19403,PA,Aldi Food Market,v-1595620733
2,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,5cde7c3f01bc5a00398f9fd4,197 E DeKalb Pike Ste 300,US,King of Prussia,United States,NaN,13396,"[197 E DeKalb Pike Ste 300, King of Prussia, P...","[{'label': 'display', 'lat': 40.0952221, 'lng'...",40.095222,-75.359016,19406,PA,ALDI,v-1595620733
3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,50bfb1c7e4b0097254152350,190 W Lincoln Hwy,US,Exton,United States,NaN,11134,"[190 W Lincoln Hwy, Exton, PA 19341, United St...","[{'label': 'display', 'lat': 40.02690649945701...",40.026906,-75.632293,19341,PA,ALDI,v-1595620733
4,[],False,4dfa9623e4cd567936067037,NaN,US,Devon,United States,NaN,7929,"[Devon, PA 19333, United States]","[{'label': 'display', 'lat': 40.0474535, 'lng'...",40.047454,-75.413013,19333,PA,Aldi,v-1595620733


In [38]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,ALDI,Grocery Store,11 Matthews Rd,US,Malvern,United States,NaN,1499,"[11 Matthews Rd, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.05055667, 'lng...",40.050557,-75.523342,19355,PA,5d0681096612700023aa5258
1,Aldi Food Market,Grocery Store,550 S Trooper Rd,US,Norristown,United States,at Egypt Rd,12537,"[550 S Trooper Rd (at Egypt Rd), Norristown, P...","[{'label': 'display', 'lat': 40.1293825, 'lng'...",40.129382,-75.398713,19403,PA,4d87bc1f7d4c5481a1723271
2,ALDI,Grocery Store,197 E DeKalb Pike Ste 300,US,King of Prussia,United States,NaN,13396,"[197 E DeKalb Pike Ste 300, King of Prussia, P...","[{'label': 'display', 'lat': 40.0952221, 'lng'...",40.095222,-75.359016,19406,PA,5cde7c3f01bc5a00398f9fd4
3,ALDI,Grocery Store,190 W Lincoln Hwy,US,Exton,United States,NaN,11134,"[190 W Lincoln Hwy, Exton, PA 19341, United St...","[{'label': 'display', 'lat': 40.02690649945701...",40.026906,-75.632293,19341,PA,50bfb1c7e4b0097254152350
4,Aldi,None,NaN,US,Devon,United States,NaN,7929,"[Devon, PA 19333, United States]","[{'label': 'display', 'lat': 40.0474535, 'lng'...",40.047454,-75.413013,19333,PA,4dfa9623e4cd567936067037


In [39]:

dataframe_filtered.shape

(5, 15)

In [40]:
df_aldi_pa = dataframe_filtered[dataframe_filtered.categories != 'Miscellaneous Shop']
df_aldi_pa.reset_index(inplace=True, drop=True)
df_aldi_pa.at[0:,'categories']='Grocery Store'
df_aldi_pa

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,ALDI,Grocery Store,11 Matthews Rd,US,Malvern,United States,NaN,1499,"[11 Matthews Rd, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.05055667, 'lng...",40.050557,-75.523342,19355,PA,5d0681096612700023aa5258
1,Aldi Food Market,Grocery Store,550 S Trooper Rd,US,Norristown,United States,at Egypt Rd,12537,"[550 S Trooper Rd (at Egypt Rd), Norristown, P...","[{'label': 'display', 'lat': 40.1293825, 'lng'...",40.129382,-75.398713,19403,PA,4d87bc1f7d4c5481a1723271
2,ALDI,Grocery Store,197 E DeKalb Pike Ste 300,US,King of Prussia,United States,NaN,13396,"[197 E DeKalb Pike Ste 300, King of Prussia, P...","[{'label': 'display', 'lat': 40.0952221, 'lng'...",40.095222,-75.359016,19406,PA,5cde7c3f01bc5a00398f9fd4
3,ALDI,Grocery Store,190 W Lincoln Hwy,US,Exton,United States,NaN,11134,"[190 W Lincoln Hwy, Exton, PA 19341, United St...","[{'label': 'display', 'lat': 40.02690649945701...",40.026906,-75.632293,19341,PA,50bfb1c7e4b0097254152350
4,Aldi,Grocery Store,NaN,US,Devon,United States,NaN,7929,"[Devon, PA 19333, United States]","[{'label': 'display', 'lat': 40.0474535, 'lng'...",40.047454,-75.413013,19333,PA,4dfa9623e4cd567936067037


In [41]:
df_aldi_pa.shape

(5, 15)

# Gas Station

In [42]:
search_query = 'Gas'
radius = 16093
print('done')

done


In [43]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Gas&radius=16093&limit=500'

In [44]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3ce8fcc0f268b53bd39a'},
 'response': {'venues': [{'id': '503fcecee4b0374347b00901',
    'name': "BJ's Gas Station",
    'location': {'address': '200-C Mill Rd.',
     'crossStreet': "outside BJ's Wholesale Club",
     'lat': 40.12687040433773,
     'lng': -75.45709823554397,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.12687040433773,
       'lng': -75.45709823554397}],
     'distance': 9297,
     'postalCode': '19460',
     'cc': 'US',
     'city': 'Oaks',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ["200-C Mill Rd. (outside BJ's Wholesale Club)",
      'Oaks, PA 19460',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d113951735',
      'name': 'Gas Station',
      'pluralName': 'Gas Stations',
      'shortName': 'Gas Station',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gas_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1

In [45]:

venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",False,503fcecee4b0374347b00901,200-C Mill Rd.,US,Oaks,United States,outside BJ's Wholesale Club,9297,"[200-C Mill Rd. (outside BJ's Wholesale Club),...","[{'label': 'display', 'lat': 40.12687040433773...",40.126870,-75.457098,19460,PA,BJ's Gas Station,v-1595620733,NaN
1,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",False,4f43b28a19834bc91f585360,108 Lancaster Ave,US,Malvern,United States,NaN,2426,"[108 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.042294, 'lng':...",40.042294,-75.531269,19355,PA,Gas Works,v-1595620733,NaN
2,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",False,4b5c5c69f964a520722c29e3,3710 West Chester Pike,US,Newtown Square,United States,at Bishop Hollow Rd,11475,"[3710 West Chester Pike (at Bishop Hollow Rd),...","[{'label': 'display', 'lat': 39.98661818420603...",39.986618,-75.401964,19073,PA,Liberty Gas Station,v-1595620733,NaN
3,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",False,4dc2de25b0fb490416b97b97,455 W Dekalb Pike,US,King of Prussia,United States,NaN,11917,"[455 W Dekalb Pike, King of Prussia, PA 19406,...","[{'label': 'display', 'lat': 40.09047299525029...",40.090473,-75.375248,19406,PA,US Petro Gas,v-1595620733,NaN
4,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,4b167d93f964a520a3b923e3,"5 Matthews Rd,",US,Malvern,United States,at Morehall Rd,1614,"[5 Matthews Rd, (at Morehall Rd), Malvern, PA ...","[{'label': 'display', 'lat': 40.050528, 'lng':...",40.050528,-75.524695,19355,PA,Wawa,v-1595620733,NaN
5,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",False,4ee5eed25c5cfa48fbcb6d89,210 W Main St,US,Norristown,United States,NaN,15200,"[210 W Main St, Norristown, PA 19401, United S...","[{'label': 'display', 'lat': 40.11561965942383...",40.115620,-75.347847,19401,PA,CITGO Gas Station,v-1595620733,NaN
6,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",False,4e2de29bc65b80dfd831877e,700 Nutt Rd,US,Phoenixville,United States,NaN,9335,"[700 Nutt Rd, Phoenixville, PA 19460, United S...","[{'label': 'display', 'lat': 40.13299905102642...",40.132999,-75.534999,19460,PA,Giant Gasoline,v-1595620733,NaN
7,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",False,4d9e2b35474d721ea864dd42,512 E King Rd,US,Malvern,United States,NaN,1406,"[512 E King Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.04091689, 'lng...",40.040917,-75.498305,19355,PA,BP,v-1595620733,574890283
8,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",False,4c844ac16e8c95213ffa6b64,1100 E Main St,US,Norristown,United States,at Ross St.,16663,"[1100 E Main St (at Ross St.), Norristown, PA ...","[{'label': 'display', 'lat': 40.10832977586617...",40.108330,-75.324538,19401,PA,Sunoco Gas Station,v-1595620733,NaN
9,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",False,4c6d3342e13db60c0a01d8b1,141 Lancaster Ave,US,Devon,United States,at Waterloo Rd.,7115,"[141 Lancaster Ave (at Waterloo Rd.), Devon, P...","[{'label': 'display', 'lat': 40.046301, 'lng':...",40.046301,-75.422723,19333,PA,Liberty Gas,v-1595620733,NaN


In [46]:

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,BJ's Gas Station,Gas Station,200-C Mill Rd.,US,Oaks,United States,outside BJ's Wholesale Club,9297,"[200-C Mill Rd. (outside BJ's Wholesale Club),...","[{'label': 'display', 'lat': 40.12687040433773...",40.126870,-75.457098,19460,PA,503fcecee4b0374347b00901
1,Gas Works,Construction & Landscaping,108 Lancaster Ave,US,Malvern,United States,NaN,2426,"[108 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.042294, 'lng':...",40.042294,-75.531269,19355,PA,4f43b28a19834bc91f585360
2,Liberty Gas Station,Gas Station,3710 West Chester Pike,US,Newtown Square,United States,at Bishop Hollow Rd,11475,"[3710 West Chester Pike (at Bishop Hollow Rd),...","[{'label': 'display', 'lat': 39.98661818420603...",39.986618,-75.401964,19073,PA,4b5c5c69f964a520722c29e3
3,US Petro Gas,Gas Station,455 W Dekalb Pike,US,King of Prussia,United States,NaN,11917,"[455 W Dekalb Pike, King of Prussia, PA 19406,...","[{'label': 'display', 'lat': 40.09047299525029...",40.090473,-75.375248,19406,PA,4dc2de25b0fb490416b97b97
4,Wawa,Convenience Store,"5 Matthews Rd,",US,Malvern,United States,at Morehall Rd,1614,"[5 Matthews Rd, (at Morehall Rd), Malvern, PA ...","[{'label': 'display', 'lat': 40.050528, 'lng':...",40.050528,-75.524695,19355,PA,4b167d93f964a520a3b923e3
5,CITGO Gas Station,Gas Station,210 W Main St,US,Norristown,United States,NaN,15200,"[210 W Main St, Norristown, PA 19401, United S...","[{'label': 'display', 'lat': 40.11561965942383...",40.115620,-75.347847,19401,PA,4ee5eed25c5cfa48fbcb6d89
6,Giant Gasoline,Gas Station,700 Nutt Rd,US,Phoenixville,United States,NaN,9335,"[700 Nutt Rd, Phoenixville, PA 19460, United S...","[{'label': 'display', 'lat': 40.13299905102642...",40.132999,-75.534999,19460,PA,4e2de29bc65b80dfd831877e
7,BP,Gas Station,512 E King Rd,US,Malvern,United States,NaN,1406,"[512 E King Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.04091689, 'lng...",40.040917,-75.498305,19355,PA,4d9e2b35474d721ea864dd42
8,Sunoco Gas Station,Gas Station,1100 E Main St,US,Norristown,United States,at Ross St.,16663,"[1100 E Main St (at Ross St.), Norristown, PA ...","[{'label': 'display', 'lat': 40.10832977586617...",40.108330,-75.324538,19401,PA,4c844ac16e8c95213ffa6b64
9,Liberty Gas,Gas Station,141 Lancaster Ave,US,Devon,United States,at Waterloo Rd.,7115,"[141 Lancaster Ave (at Waterloo Rd.), Devon, P...","[{'label': 'display', 'lat': 40.046301, 'lng':...",40.046301,-75.422723,19333,PA,4c6d3342e13db60c0a01d8b1


In [47]:

dataframe_filtered.shape

(50, 15)

In [48]:
df_gas_pa = dataframe_filtered[dataframe_filtered.categories == 'Gas Station']
df_gas_pa.reset_index(inplace=True, drop=True)
df_gas_pa

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,BJ's Gas Station,Gas Station,200-C Mill Rd.,US,Oaks,United States,outside BJ's Wholesale Club,9297,"[200-C Mill Rd. (outside BJ's Wholesale Club),...","[{'label': 'display', 'lat': 40.12687040433773...",40.126870,-75.457098,19460,PA,503fcecee4b0374347b00901
1,Liberty Gas Station,Gas Station,3710 West Chester Pike,US,Newtown Square,United States,at Bishop Hollow Rd,11475,"[3710 West Chester Pike (at Bishop Hollow Rd),...","[{'label': 'display', 'lat': 39.98661818420603...",39.986618,-75.401964,19073,PA,4b5c5c69f964a520722c29e3
2,US Petro Gas,Gas Station,455 W Dekalb Pike,US,King of Prussia,United States,NaN,11917,"[455 W Dekalb Pike, King of Prussia, PA 19406,...","[{'label': 'display', 'lat': 40.09047299525029...",40.090473,-75.375248,19406,PA,4dc2de25b0fb490416b97b97
3,CITGO Gas Station,Gas Station,210 W Main St,US,Norristown,United States,NaN,15200,"[210 W Main St, Norristown, PA 19401, United S...","[{'label': 'display', 'lat': 40.11561965942383...",40.115620,-75.347847,19401,PA,4ee5eed25c5cfa48fbcb6d89
4,Giant Gasoline,Gas Station,700 Nutt Rd,US,Phoenixville,United States,NaN,9335,"[700 Nutt Rd, Phoenixville, PA 19460, United S...","[{'label': 'display', 'lat': 40.13299905102642...",40.132999,-75.534999,19460,PA,4e2de29bc65b80dfd831877e
5,BP,Gas Station,512 E King Rd,US,Malvern,United States,NaN,1406,"[512 E King Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.04091689, 'lng...",40.040917,-75.498305,19355,PA,4d9e2b35474d721ea864dd42
6,Sunoco Gas Station,Gas Station,1100 E Main St,US,Norristown,United States,at Ross St.,16663,"[1100 E Main St (at Ross St.), Norristown, PA ...","[{'label': 'display', 'lat': 40.10832977586617...",40.108330,-75.324538,19401,PA,4c844ac16e8c95213ffa6b64
7,Liberty Gas,Gas Station,141 Lancaster Ave,US,Devon,United States,at Waterloo Rd.,7115,"[141 Lancaster Ave (at Waterloo Rd.), Devon, P...","[{'label': 'display', 'lat': 40.046301, 'lng':...",40.046301,-75.422723,19333,PA,4c6d3342e13db60c0a01d8b1
8,Pioneer Gas,Gas Station,NaN,US,East Whiteland Township,United States,rt 30 across frazer wawa,5813,"[rt 30 across frazer wawa, East Whiteland Town...","[{'label': 'display', 'lat': 40.0353546, 'lng'...",40.035355,-75.570459,19355,PA,4bddcb700ee3a5933dac2eb0
9,Halman's Gulf Gas Station,Gas Station,1600 Pikeland Rd,US,Chester Springs,United States,NaN,10529,"[1600 Pikeland Rd, Chester Springs, PA 19425, ...","[{'label': 'display', 'lat': 40.10035900000000...",40.100359,-75.612220,19425,PA,4d0e59097d28721ebbc2fd20


In [49]:
df_gas_pa.shape

(29, 15)

# Park

In [50]:
search_query = 'Park'
radius = 16093
print('done')

done


In [51]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Park&radius=16093&limit=500'

In [52]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5f1b3bc2cbd37754cccb1079'},
 'response': {'venues': [{'id': '4b9fb698f964a520873737e3',
    'name': 'Wilson Farm Park',
    'location': {'address': '600 Lee Rd',
     'crossStreet': 'Chesterbrook blvd',
     'lat': 40.070877368891104,
     'lng': -75.45433832035536,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.070877368891104,
       'lng': -75.45433832035536}],
     'distance': 4860,
     'postalCode': '19087',
     'cc': 'US',
     'city': 'Chesterbrook',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['600 Lee Rd (Chesterbrook blvd)',
      'Chesterbrook, PA 19087',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'shortName': 'Park',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595620734',
    'hasP

In [53]:

venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4b9fb698f964a520873737e3,600 Lee Rd,US,Chesterbrook,United States,Chesterbrook blvd,4860,"[600 Lee Rd (Chesterbrook blvd), Chesterbrook,...","[{'label': 'display', 'lat': 40.07087736889110...",40.070877,-75.454338,19087,PA,Wilson Farm Park,v-1595620734,NaN
1,"[{'id': '52e81612bcbc57f1066b7a21', 'name': 'N...",False,4abd7e28f964a520a78a20e3,1400 N Outer Line Dr,US,King of Prussia,United States,btwn Gulph & Valley Creek Rd,8858,[1400 N Outer Line Dr (btwn Gulph & Valley Cre...,"[{'label': 'display', 'lat': 40.10143382738081...",40.101434,-75.424216,19406,PA,Valley Forge National Historical Park,v-1595620734,NaN
2,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4caf65fedb32f04d291ab74d,Monument Ave & Malvern Ave,US,Malvern,United States,at Malvern Memorial Park,2927,[Monument Ave & Malvern Ave (at Malvern Memori...,"[{'label': 'display', 'lat': 40.02994601580231...",40.029946,-75.524272,19355,PA,Malvern Memorial Park Playground,v-1595620734,NaN
3,"[{'id': '5bae9231bedf3950379f89d0', 'name': 'S...",False,4a9abbaef964a520653220e3,1023 Sycamore Mills Rd,US,Media,United States,NaN,12167,"[1023 Sycamore Mills Rd, Media, PA 19063, Unit...","[{'label': 'display', 'lat': 39.95281296856593...",39.952813,-75.446334,19063,PA,Ridley Creek State Park,v-1595620734,NaN
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4b93e048f964a520cd5634e3,1661 Paoli Pike,US,West Chester,United States,NaN,6439,"[1661 Paoli Pike, West Chester, PA 19380, Unit...","[{'label': 'display', 'lat': 40.00112555094431...",40.001126,-75.541456,19380,PA,East Goshen Park,v-1595620734,NaN
5,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4bd0a343046076b02b627071,1200 Wilson Dr,US,West Chester,United States,at Airport Rd,9363,"[1200 Wilson Dr (at Airport Rd), West Chester,...","[{'label': 'display', 'lat': 39.99305312665248...",39.993053,-75.584033,19380,PA,QVC Studio Park,v-1595620734,NaN
6,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",False,515cb021e4b06e03082ed13f,2 Industrial Blvd,US,Paoli,United States,NaN,1202,"[2 Industrial Blvd, Paoli, PA 19301, United St...","[{'label': 'display', 'lat': 40.04308700561523...",40.043087,-75.498199,19301,PA,Paoli Office Park,v-1595620734,NaN
7,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4fc0138dbb3dbdc3563ba687,1235 Westlakes Dr,US,Berwyn,United States,Wsedesford Rd.,3190,"[1235 Westlakes Dr (Wsedesford Rd.), Berwyn, P...","[{'label': 'display', 'lat': 40.063609, 'lng':...",40.063609,-75.471550,19312,PA,"Mack-Cali, One Westlakes, Westlakes Office Park",v-1595620734,NaN
8,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4fc037a4bb3dbdc3564160d2,1205 Westlakes Dr,US,Berwyn,United States,W. Swedesford Rd.,3348,"[1205 Westlakes Dr (W. Swedesford Rd.), Berwyn...","[{'label': 'display', 'lat': 40.06417977262549...",40.064180,-75.469851,19312,PA,Mack-Cali-Two Westlakes Office Park,v-1595620734,NaN
9,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4fdf94d3c59367dba34ea77a,1000 Westlakes Dr,US,Berwyn,United States,NaN,3414,"[1000 Westlakes Dr, Berwyn, PA 19312, United S...","[{'label': 'display', 'lat': 40.061552, 'lng':...",40.061552,-75.467729,19312,PA,Mack-Cali Westlakes Office Park,v-1595620734,NaN


In [54]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Wilson Farm Park,Park,600 Lee Rd,US,Chesterbrook,United States,Chesterbrook blvd,4860,"[600 Lee Rd (Chesterbrook blvd), Chesterbrook,...","[{'label': 'display', 'lat': 40.07087736889110...",40.070877,-75.454338,19087,PA,4b9fb698f964a520873737e3
1,Valley Forge National Historical Park,National Park,1400 N Outer Line Dr,US,King of Prussia,United States,btwn Gulph & Valley Creek Rd,8858,[1400 N Outer Line Dr (btwn Gulph & Valley Cre...,"[{'label': 'display', 'lat': 40.10143382738081...",40.101434,-75.424216,19406,PA,4abd7e28f964a520a78a20e3
2,Malvern Memorial Park Playground,Playground,Monument Ave & Malvern Ave,US,Malvern,United States,at Malvern Memorial Park,2927,[Monument Ave & Malvern Ave (at Malvern Memori...,"[{'label': 'display', 'lat': 40.02994601580231...",40.029946,-75.524272,19355,PA,4caf65fedb32f04d291ab74d
3,Ridley Creek State Park,State / Provincial Park,1023 Sycamore Mills Rd,US,Media,United States,NaN,12167,"[1023 Sycamore Mills Rd, Media, PA 19063, Unit...","[{'label': 'display', 'lat': 39.95281296856593...",39.952813,-75.446334,19063,PA,4a9abbaef964a520653220e3
4,East Goshen Park,Park,1661 Paoli Pike,US,West Chester,United States,NaN,6439,"[1661 Paoli Pike, West Chester, PA 19380, Unit...","[{'label': 'display', 'lat': 40.00112555094431...",40.001126,-75.541456,19380,PA,4b93e048f964a520cd5634e3
5,QVC Studio Park,Office,1200 Wilson Dr,US,West Chester,United States,at Airport Rd,9363,"[1200 Wilson Dr (at Airport Rd), West Chester,...","[{'label': 'display', 'lat': 39.99305312665248...",39.993053,-75.584033,19380,PA,4bd0a343046076b02b627071
6,Paoli Office Park,Medical Center,2 Industrial Blvd,US,Paoli,United States,NaN,1202,"[2 Industrial Blvd, Paoli, PA 19301, United St...","[{'label': 'display', 'lat': 40.04308700561523...",40.043087,-75.498199,19301,PA,515cb021e4b06e03082ed13f
7,"Mack-Cali, One Westlakes, Westlakes Office Park",Building,1235 Westlakes Dr,US,Berwyn,United States,Wsedesford Rd.,3190,"[1235 Westlakes Dr (Wsedesford Rd.), Berwyn, P...","[{'label': 'display', 'lat': 40.063609, 'lng':...",40.063609,-75.471550,19312,PA,4fc0138dbb3dbdc3563ba687
8,Mack-Cali-Two Westlakes Office Park,Building,1205 Westlakes Dr,US,Berwyn,United States,W. Swedesford Rd.,3348,"[1205 Westlakes Dr (W. Swedesford Rd.), Berwyn...","[{'label': 'display', 'lat': 40.06417977262549...",40.064180,-75.469851,19312,PA,4fc037a4bb3dbdc3564160d2
9,Mack-Cali Westlakes Office Park,Building,1000 Westlakes Dr,US,Berwyn,United States,NaN,3414,"[1000 Westlakes Dr, Berwyn, PA 19312, United S...","[{'label': 'display', 'lat': 40.061552, 'lng':...",40.061552,-75.467729,19312,PA,4fdf94d3c59367dba34ea77a


In [55]:
dataframe_filtered.shape

(50, 15)

In [56]:
df1 = dataframe_filtered[dataframe_filtered.categories != 'Building']
df2 = df1[df1.categories != 'Medical Center']
df3 = df2[df2.categories != 'Office']
df4 = df3[df3.categories != 'Parking']
df5 = df4[df4.categories != 'Business Center']
df6 = df5[df5.categories != 'Intersection']
df7 = df6[df6.categories != 'Residential Building (Apartment / Condo)']
df8 = df7[df7.categories != 'Light Rail Station']
df9 = df8[df8.categories != 'College Science Building']
df9.reset_index(inplace=True, drop=True)
df_park_pa = df9
df_park_pa.at[0:, 'categories']= 'Park'

df_park_pa


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Wilson Farm Park,Park,600 Lee Rd,US,Chesterbrook,United States,Chesterbrook blvd,4860,"[600 Lee Rd (Chesterbrook blvd), Chesterbrook,...","[{'label': 'display', 'lat': 40.07087736889110...",40.070877,-75.454338,19087,PA,4b9fb698f964a520873737e3
1,Valley Forge National Historical Park,Park,1400 N Outer Line Dr,US,King of Prussia,United States,btwn Gulph & Valley Creek Rd,8858,[1400 N Outer Line Dr (btwn Gulph & Valley Cre...,"[{'label': 'display', 'lat': 40.10143382738081...",40.101434,-75.424216,19406,PA,4abd7e28f964a520a78a20e3
2,Malvern Memorial Park Playground,Park,Monument Ave & Malvern Ave,US,Malvern,United States,at Malvern Memorial Park,2927,[Monument Ave & Malvern Ave (at Malvern Memori...,"[{'label': 'display', 'lat': 40.02994601580231...",40.029946,-75.524272,19355,PA,4caf65fedb32f04d291ab74d
3,Ridley Creek State Park,Park,1023 Sycamore Mills Rd,US,Media,United States,NaN,12167,"[1023 Sycamore Mills Rd, Media, PA 19063, Unit...","[{'label': 'display', 'lat': 39.95281296856593...",39.952813,-75.446334,19063,PA,4a9abbaef964a520653220e3
4,East Goshen Park,Park,1661 Paoli Pike,US,West Chester,United States,NaN,6439,"[1661 Paoli Pike, West Chester, PA 19380, Unit...","[{'label': 'display', 'lat': 40.00112555094431...",40.001126,-75.541456,19380,PA,4b93e048f964a520cd5634e3
5,Valley Creek Park,Park,361 N Morehall Rd,US,Malvern,United States,NaN,2411,"[361 N Morehall Rd, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.06060142145336...",40.060601,-75.531932,19355,PA,4b61ea4ff964a520042b2ae3
6,Tredyffrin Township Park,Park,NaN,US,Paoli,United States,NaN,949,"[Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.043804, 'lng':...",40.043804,-75.508165,19301,PA,50d79781e4b0e25340b59b5f
7,Artillery Park,Park,5450 E Inner Line Dr,US,King of Prussia,United States,at Valley Forge National Historical Park,6771,[5450 E Inner Line Dr (at Valley Forge Nationa...,"[{'label': 'display', 'lat': 40.09367078505359...",40.093671,-75.447769,19046,PA,4e9ae9bb02d5df1b506be813
8,Park Station - Port Kennedy,Park,Station Rd/,US,King of Prussia,United States,at Valley Forge National Historical Park,9628,[Station Rd/ (at Valley Forge National Histori...,"[{'label': 'display', 'lat': 40.10624864810545...",40.106249,-75.417663,19046,PA,4bdb1552c79cc928480c81e9
9,Reynolds Dog Park,Park,Longford Rd.,US,Upper Providence,United States,NaN,8851,"[Longford Rd., Upper Providence, PA 19460, Uni...","[{'label': 'display', 'lat': 40.12965783770706...",40.129658,-75.482686,19460,PA,4ba14c7cf964a52056aa37e3


In [57]:
df_park_pa.shape

(38, 15)

# Shopping Mall

In [58]:

search_query = 'Shopping Mall'
radius = 16093
print('done')

done


In [59]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Shopping Mall&radius=16093&limit=500'

In [60]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3a5f0c15405063830750'},
 'response': {'venues': [{'id': '49efbd7cf964a520ff681fe3',
    'name': 'King of Prussia Mall',
    'location': {'address': '160 N Gulph Rd',
     'crossStreet': 'at Mall Blvd',
     'lat': 40.0883703,
     'lng': -75.3915726,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.0883703,
       'lng': -75.3915726}],
     'distance': 10538,
     'postalCode': '19406',
     'cc': 'US',
     'city': 'King of Prussia',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['160 N Gulph Rd (at Mall Blvd)',
      'King of Prussia, PA 19406',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595620734',
    'hasPerk': False}

In [61]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,49efbd7cf964a520ff681fe3,160 N Gulph Rd,...,10538,"[160 N Gulph Rd (at Mall Blvd), King of Prussi...","[{'label': 'display', 'lat': 40.0883703, 'lng'...",40.088370,-75.391573,19406,PA,King of Prussia Mall,v-1595620734,NaN
1,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,52962cca11d2c01eb363487f,King Street,...,2049,"[King Street, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03555199844064...",40.035552,-75.516282,19355,PA,Malvern Shopping Center,v-1595620734,NaN
2,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4e3439b488772aabd8b52b28,82 E Lancaster Ave,...,2707,"[82 E Lancaster Ave, Paoli, PA 19301, United S...","[{'label': 'display', 'lat': 40.04084001432724...",40.040840,-75.477748,19301,PA,Paoli Shopping Center,v-1595620734,NaN
3,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,539391a3498ea919cfa050ab,81 Lancaster Ave,...,2231,"[81 Lancaster Ave, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.044354, 'lng':...",40.044354,-75.529999,19355,PA,Great Valley Shopping Center,v-1595620734,NaN
4,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,563f986acd10e8b144332a2e,5 Morehall Rd,...,1658,"[5 Morehall Rd (at W Lancaster Ave), Malvern, ...","[{'label': 'display', 'lat': 40.04459762573242...",40.044598,-75.522644,19355,PA,Malvern Courtyard Shopping Center,v-1595620734,NaN
5,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b0c6fa1f964a5203b3d23e3,225 Lancaster Ave,...,3265,"[225 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.04230206350306...",40.042302,-75.541968,19355,PA,GIANT Food Stores,v-1595620734,NaN
6,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,51081f93e4b0d4a062abbd05,32 W Lancaster Ave,...,2313,"[32 W Lancaster Ave, Paoli, PA 19301, United S...","[{'label': 'display', 'lat': 40.04166030883789...",40.041660,-75.482430,19301,PA,Paoli West Shopping Center,v-1595620734,NaN
7,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ed76f509a52145470d72120,125 E Swedesford Rd,...,6620,"[125 E Swedesford Rd, Wayne, PA 19087, United ...","[{'label': 'display', 'lat': 40.073188, 'lng':...",40.073188,-75.433183,19087,PA,Gateway Shopping Center,v-1595620734,NaN
8,"[{'id': '54541900498ea6ccd0202697', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,5d1bf02aaa7be7002a4e3dfe,82 East Lancaster Avenue,...,2702,"[82 East Lancaster Avenue, Paoli, PA 19301, Un...","[{'label': 'display', 'lat': 40.04144743, 'lng...",40.041447,-75.477403,19301,PA,The Lash Lounge,v-1595620734,555924786
9,"[{'id': '5453de49498eade8af355881', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,57e224d6498eb1f1b7f65900,109 Spring Rd,...,2603,"[109 Spring Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.037743, 'lng':...",40.037743,-75.529942,19355,PA,"O'Malley Topsoil, LLC",v-1595620734,NaN


In [62]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,King of Prussia Mall,Shopping Mall,160 N Gulph Rd,US,King of Prussia,United States,at Mall Blvd,10538,"[160 N Gulph Rd (at Mall Blvd), King of Prussi...","[{'label': 'display', 'lat': 40.0883703, 'lng'...",40.088370,-75.391573,19406,PA,49efbd7cf964a520ff681fe3
1,Malvern Shopping Center,Shopping Plaza,King Street,US,Malvern,United States,NaN,2049,"[King Street, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03555199844064...",40.035552,-75.516282,19355,PA,52962cca11d2c01eb363487f
2,Paoli Shopping Center,Shopping Plaza,82 E Lancaster Ave,US,Paoli,United States,NaN,2707,"[82 E Lancaster Ave, Paoli, PA 19301, United S...","[{'label': 'display', 'lat': 40.04084001432724...",40.040840,-75.477748,19301,PA,4e3439b488772aabd8b52b28
3,Great Valley Shopping Center,Shopping Plaza,81 Lancaster Ave,US,Malvern,United States,NaN,2231,"[81 Lancaster Ave, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.044354, 'lng':...",40.044354,-75.529999,19355,PA,539391a3498ea919cfa050ab
4,Malvern Courtyard Shopping Center,Shopping Plaza,5 Morehall Rd,US,Malvern,United States,at W Lancaster Ave,1658,"[5 Morehall Rd (at W Lancaster Ave), Malvern, ...","[{'label': 'display', 'lat': 40.04459762573242...",40.044598,-75.522644,19355,PA,563f986acd10e8b144332a2e
5,GIANT Food Stores,Supermarket,225 Lancaster Ave,US,Malvern,United States,NaN,3265,"[225 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.04230206350306...",40.042302,-75.541968,19355,PA,4b0c6fa1f964a5203b3d23e3
6,Paoli West Shopping Center,Shopping Plaza,32 W Lancaster Ave,US,Paoli,United States,NaN,2313,"[32 W Lancaster Ave, Paoli, PA 19301, United S...","[{'label': 'display', 'lat': 40.04166030883789...",40.041660,-75.482430,19301,PA,51081f93e4b0d4a062abbd05
7,Gateway Shopping Center,Shopping Plaza,125 E Swedesford Rd,US,Wayne,United States,NaN,6620,"[125 E Swedesford Rd, Wayne, PA 19087, United ...","[{'label': 'display', 'lat': 40.073188, 'lng':...",40.073188,-75.433183,19087,PA,4ed76f509a52145470d72120
8,The Lash Lounge,Health & Beauty Service,82 East Lancaster Avenue,US,Paoli,United States,NaN,2702,"[82 East Lancaster Avenue, Paoli, PA 19301, Un...","[{'label': 'display', 'lat': 40.04144743, 'lng...",40.041447,-75.477403,19301,PA,5d1bf02aaa7be7002a4e3dfe
9,"O'Malley Topsoil, LLC",Business Service,109 Spring Rd,US,Malvern,United States,NaN,2603,"[109 Spring Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.037743, 'lng':...",40.037743,-75.529942,19355,PA,57e224d6498eb1f1b7f65900


In [63]:
dataframe_filtered.shape

(50, 15)

In [64]:
df_mall_pa = dataframe_filtered[(dataframe_filtered['categories']=='Shopping Mall') | (dataframe_filtered['categories'] =='Shopping Plaza')]
df_mall_pa.reset_index(inplace=True, drop=True)

df_mall_pa.at[0:, 'categories']='Shopping Mall'
df_mall_pa

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,King of Prussia Mall,Shopping Mall,160 N Gulph Rd,US,King of Prussia,United States,at Mall Blvd,10538,"[160 N Gulph Rd (at Mall Blvd), King of Prussi...","[{'label': 'display', 'lat': 40.0883703, 'lng'...",40.088370,-75.391573,19406,PA,49efbd7cf964a520ff681fe3
1,Malvern Shopping Center,Shopping Mall,King Street,US,Malvern,United States,NaN,2049,"[King Street, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03555199844064...",40.035552,-75.516282,19355,PA,52962cca11d2c01eb363487f
2,Paoli Shopping Center,Shopping Mall,82 E Lancaster Ave,US,Paoli,United States,NaN,2707,"[82 E Lancaster Ave, Paoli, PA 19301, United S...","[{'label': 'display', 'lat': 40.04084001432724...",40.040840,-75.477748,19301,PA,4e3439b488772aabd8b52b28
3,Great Valley Shopping Center,Shopping Mall,81 Lancaster Ave,US,Malvern,United States,NaN,2231,"[81 Lancaster Ave, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.044354, 'lng':...",40.044354,-75.529999,19355,PA,539391a3498ea919cfa050ab
4,Malvern Courtyard Shopping Center,Shopping Mall,5 Morehall Rd,US,Malvern,United States,at W Lancaster Ave,1658,"[5 Morehall Rd (at W Lancaster Ave), Malvern, ...","[{'label': 'display', 'lat': 40.04459762573242...",40.044598,-75.522644,19355,PA,563f986acd10e8b144332a2e
5,Paoli West Shopping Center,Shopping Mall,32 W Lancaster Ave,US,Paoli,United States,NaN,2313,"[32 W Lancaster Ave, Paoli, PA 19301, United S...","[{'label': 'display', 'lat': 40.04166030883789...",40.041660,-75.482430,19301,PA,51081f93e4b0d4a062abbd05
6,Gateway Shopping Center,Shopping Mall,125 E Swedesford Rd,US,Wayne,United States,NaN,6620,"[125 E Swedesford Rd, Wayne, PA 19087, United ...","[{'label': 'display', 'lat': 40.073188, 'lng':...",40.073188,-75.433183,19087,PA,4ed76f509a52145470d72120
7,Frazer View Shopping Center,Shopping Mall,365 Lancaster Ave,US,Frazer,United States,NaN,4343,"[365 Lancaster Ave, Frazer, PA 19355, United S...","[{'label': 'display', 'lat': 40.03873768715734...",40.038738,-75.553735,19355,PA,5033804fe4b020d49f0a1550
8,Chesterbrook Village Shopping Center,Shopping Mall,500 Chesterbrook Blvd,US,Chesterbrook,United States,NaN,4829,"[500 Chesterbrook Blvd, Chesterbrook, PA 19087...","[{'label': 'display', 'lat': 40.074061, 'lng':...",40.074061,-75.456950,19087,PA,5a28413ac97f287a861a4edd
9,Exton Square Mall,Shopping Mall,260 Exton Square Mall,US,Exton,United States,at US 30 & Rt 100,10322,"[260 Exton Square Mall (at US 30 & Rt 100), Ex...","[{'label': 'display', 'lat': 40.03152487507484...",40.031525,-75.623960,19341,PA,4ac17398f964a520359720e3


In [65]:
df_mall_pa.shape

(20, 15)

# Thai Restaurant

In [66]:
search_query = 'Thai Restaurant'
radius = 16093
print('done')

done


In [67]:

url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Thai Restaurant&radius=16093&limit=500'

In [68]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3b88588ea57d027316dd'},
 'response': {'venues': [{'id': '4f32838c19836c91c7df31a2',
    'name': 'Sabai Thai Restaurant',
    'location': {'address': '3707 West Chester Pike',
     'lat': 39.987205505371094,
     'lng': -75.40178680419922,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.987205505371094,
       'lng': -75.40178680419922}],
     'distance': 11445,
     'postalCode': '19073',
     'cc': 'US',
     'city': 'Newtown Square',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['3707 West Chester Pike',
      'Newtown Square, PA 19073',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d149941735',
      'name': 'Thai Restaurant',
      'pluralName': 'Thai Restaurants',
      'shortName': 'Thai',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/thai_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595620735',
    'hasPerk': False},
 

In [69]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f32838c19836c91c7df31a2,3707 West Chester Pike,...,11445,"[3707 West Chester Pike, Newtown Square, PA 19...","[{'label': 'display', 'lat': 39.98720550537109...",39.987206,-75.401787,19073,PA,Sabai Thai Restaurant,v-1595620735,NaN
1,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ae07cbcf964a520947f21e3,556 Lancaster Ave,...,5817,"[556 Lancaster Ave, Berwyn, PA 19312, United S...","[{'label': 'display', 'lat': 40.04769522619646...",40.047695,-75.437849,19312,PA,Thai Orchid Restaurant,v-1595620735,NaN
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b6360d2f964a520a1752ae3,127 W King St,...,2084,"[127 W King St, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03535596937267...",40.035356,-75.516676,19355,PA,Anthony's Pizza & Italian Restaurant,v-1595620735,428256912
3,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbfd29ef8219c74a4bcb010,700 Nutt Rd,...,9284,"[700 Nutt Rd (at Phoenixville Plaza), Phoenixv...","[{'label': 'display', 'lat': 40.13277724570432...",40.132777,-75.533788,19460,PA,The Thai Place,v-1595620735,NaN
4,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",1937458,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/tjs-restaur...,False,4b0e1ccbf964a520215523e3,35 Paoli Plz,...,2052,"[35 Paoli Plz (at Paoli Village Shoppes), Paol...","[{'label': 'display', 'lat': 40.04230603716183...",40.042306,-75.485511,19301,PA,TJ's Restaurant and Drinkery,v-1595620735,NaN
5,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",2175462,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/royal-thai-...,False,4b833346f964a52019fd30e3,309 Lancaster Ave,...,3917,"[309 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.03979160099619...",40.039792,-75.548916,19355,PA,Royal Thai Orchid,v-1595620735,NaN
6,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4cd4af3067c7236ab8ce0e77,16 Great Valley Pkwy,...,2864,"[16 Great Valley Pkwy (at Route 29), Malvern, ...","[{'label': 'display', 'lat': 40.06218696708645...",40.062187,-75.536830,19355,PA,Twenty9 Restaurant And Bar,v-1595620735,NaN
7,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4baa465ff964a520125a3ae3,81 Lancaster Ave,...,2252,[81 Lancaster Ave (at Great Valley Shopping Ce...,"[{'label': 'display', 'lat': 40.04370974942453...",40.043710,-75.529896,19355,PA,East Star Chinese Restaurant,v-1595620735,NaN
8,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c23b1def1272d7f52d881c5,7 W King St,...,1957,"[7 W King St, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.0359731505773,...",40.035973,-75.514875,19355,PA,Restaurant Alba,v-1595620735,NaN
9,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",2164873,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/mino-japane...,False,4b363adbf964a520553125e3,321 Lancaster Ave,...,3918,"[321 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.03969343354500...",40.039693,-75.548878,19355,PA,Mino Japanese Restaurant,v-1595620735,NaN


In [70]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Sabai Thai Restaurant,Thai Restaurant,3707 West Chester Pike,US,Newtown Square,United States,NaN,11445,"[3707 West Chester Pike, Newtown Square, PA 19...","[{'label': 'display', 'lat': 39.98720550537109...",39.987206,-75.401787,19073,PA,4f32838c19836c91c7df31a2
1,Thai Orchid Restaurant,Thai Restaurant,556 Lancaster Ave,US,Berwyn,United States,NaN,5817,"[556 Lancaster Ave, Berwyn, PA 19312, United S...","[{'label': 'display', 'lat': 40.04769522619646...",40.047695,-75.437849,19312,PA,4ae07cbcf964a520947f21e3
2,Anthony's Pizza & Italian Restaurant,Italian Restaurant,127 W King St,US,Malvern,United States,NaN,2084,"[127 W King St, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03535596937267...",40.035356,-75.516676,19355,PA,4b6360d2f964a520a1752ae3
3,The Thai Place,Thai Restaurant,700 Nutt Rd,US,Phoenixville,United States,at Phoenixville Plaza,9284,"[700 Nutt Rd (at Phoenixville Plaza), Phoenixv...","[{'label': 'display', 'lat': 40.13277724570432...",40.132777,-75.533788,19460,PA,4bbfd29ef8219c74a4bcb010
4,TJ's Restaurant and Drinkery,Bar,35 Paoli Plz,US,Paoli,United States,at Paoli Village Shoppes,2052,"[35 Paoli Plz (at Paoli Village Shoppes), Paol...","[{'label': 'display', 'lat': 40.04230603716183...",40.042306,-75.485511,19301,PA,4b0e1ccbf964a520215523e3
5,Royal Thai Orchid,Thai Restaurant,309 Lancaster Ave,US,Malvern,United States,NaN,3917,"[309 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.03979160099619...",40.039792,-75.548916,19355,PA,4b833346f964a52019fd30e3
6,Twenty9 Restaurant And Bar,Bar,16 Great Valley Pkwy,US,Malvern,United States,at Route 29,2864,"[16 Great Valley Pkwy (at Route 29), Malvern, ...","[{'label': 'display', 'lat': 40.06218696708645...",40.062187,-75.536830,19355,PA,4cd4af3067c7236ab8ce0e77
7,East Star Chinese Restaurant,Chinese Restaurant,81 Lancaster Ave,US,Malvern,United States,at Great Valley Shopping Center,2252,[81 Lancaster Ave (at Great Valley Shopping Ce...,"[{'label': 'display', 'lat': 40.04370974942453...",40.043710,-75.529896,19355,PA,4baa465ff964a520125a3ae3
8,Restaurant Alba,Italian Restaurant,7 W King St,US,Malvern,United States,NaN,1957,"[7 W King St, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.0359731505773,...",40.035973,-75.514875,19355,PA,4c23b1def1272d7f52d881c5
9,Mino Japanese Restaurant,Japanese Restaurant,321 Lancaster Ave,US,Malvern,United States,NaN,3918,"[321 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.03969343354500...",40.039693,-75.548878,19355,PA,4b363adbf964a520553125e3


In [71]:
dataframe_filtered.shape

(50, 15)

In [72]:

df_thai_pa = dataframe_filtered[dataframe_filtered.categories=='Thai Restaurant']
df_thai_pa.reset_index(inplace=True, drop=True)
df_thai_pa

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Sabai Thai Restaurant,Thai Restaurant,3707 West Chester Pike,US,Newtown Square,United States,NaN,11445,"[3707 West Chester Pike, Newtown Square, PA 19...","[{'label': 'display', 'lat': 39.98720550537109...",39.987206,-75.401787,19073,PA,4f32838c19836c91c7df31a2
1,Thai Orchid Restaurant,Thai Restaurant,556 Lancaster Ave,US,Berwyn,United States,NaN,5817,"[556 Lancaster Ave, Berwyn, PA 19312, United S...","[{'label': 'display', 'lat': 40.04769522619646...",40.047695,-75.437849,19312,PA,4ae07cbcf964a520947f21e3
2,The Thai Place,Thai Restaurant,700 Nutt Rd,US,Phoenixville,United States,at Phoenixville Plaza,9284,"[700 Nutt Rd (at Phoenixville Plaza), Phoenixv...","[{'label': 'display', 'lat': 40.13277724570432...",40.132777,-75.533788,19460,PA,4bbfd29ef8219c74a4bcb010
3,Royal Thai Orchid,Thai Restaurant,309 Lancaster Ave,US,Malvern,United States,NaN,3917,"[309 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.03979160099619...",40.039792,-75.548916,19355,PA,4b833346f964a52019fd30e3
4,Thai Place Restaurant,Thai Restaurant,495 Nutt Rd,US,Phoenixville,United States,NaN,8677,"[495 Nutt Rd, Phoenixville, PA 19460, United S...","[{'label': 'display', 'lat': 40.128686, 'lng':...",40.128686,-75.525214,19460,PA,4f32b64619836c91c7f24922
5,Thai Basil Bistro,Thai Restaurant,305 Second Ave,US,Collegeville,United States,at Collegeville Shopping Center,14910,[305 Second Ave (at Collegeville Shopping Cent...,"[{'label': 'display', 'lat': 40.18020311915131...",40.180203,-75.454481,19426,PA,4b92e5ecf964a520572534e3
6,Ruby Thai Kitchen,Thai Restaurant,NaN,US,King of Prussia,United States,NaN,10390,"[King of Prussia, PA 19406, United States]","[{'label': 'display', 'lat': 40.087899, 'lng':...",40.087899,-75.393200,19406,PA,5c0c5ced5c6838002c38418f


In [73]:
df_thai_pa.shape

(7, 15)

# Museum

In [74]:

search_query = 'Museum'
radius = 16093
print('done')

done


In [75]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Museum&radius=16093&limit=500'

In [76]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3d272321bc05cd95adb8'},
 'response': {'venues': [{'id': '4bc12eacb492d13a8cbda560',
    'name': 'American Helicopter Museum',
    'location': {'address': '1220 American Blvd',
     'lat': 39.99175853514661,
     'lng': -75.57920888352511,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.99175853514661,
       'lng': -75.57920888352511},
      {'label': 'entrance', 'lat': 39.992058, 'lng': -75.57905}],
     'distance': 9180,
     'postalCode': '19380',
     'cc': 'US',
     'city': 'West Chester',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['1220 American Blvd',
      'West Chester, PA 19380',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d181941735',
      'name': 'Museum',
      'pluralName': 'Museums',
      'shortName': 'Museum',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_',
       'suffix': '.png'},
      'primary': True}],
   

In [77]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",False,4bc12eacb492d13a8cbda560,1220 American Blvd,US,West Chester,United States,NaN,9180,"[1220 American Blvd, West Chester, PA 19380, U...","[{'label': 'display', 'lat': 39.99175853514661...",39.991759,-75.579209,19380,PA,American Helicopter Museum,v-1595620736,134606459
1,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",False,4e3d8227aeb73139a179eeb7,1520 Horse Shoe Trl,US,Malvern,United States,NaN,3690,"[1520 Horse Shoe Trl, Malvern, PA 19355, Unite...","[{'label': 'display', 'lat': 40.08381813883635...",40.083818,-75.493054,19355,PA,Wharton Esherick Museum,v-1595620736,NaN
2,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",False,4c797a2a566db60c1f21450e,Newtown Square,US,Delaware County,United States,NaN,10634,"[Newtown Square, Delaware County, PA, United S...","[{'label': 'display', 'lat': 39.98138967391212...",39.981390,-75.422059,NaN,PA,Train Museum Park,v-1595620736,NaN
3,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",False,4e389b05fa763bfcc366590c,NaN,US,Newtown Square,United States,NaN,9430,"[Newtown Square, PA, United States]","[{'label': 'display', 'lat': 40.01738209287441...",40.017382,-75.404970,NaN,PA,Papermill House Museum,v-1595620736,NaN
4,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",False,4da0aa1db3e7236a9613f878,4194-4222 W Chester Pike,US,Newtown Square,United States,Drexel Lodge Park,9700,"[4194-4222 W Chester Pike (Drexel Lodge Park),...","[{'label': 'display', 'lat': 39.974042, 'lng':...",39.974042,-75.455423,19073,PA,Newtown Railroad Museum,v-1595620736,NaN
5,[],False,4f5ca513e4b0cfc5e1e4099e,NaN,US,NaN,United States,NaN,13115,"[Pennsylvania, United States]","[{'label': 'display', 'lat': 40.02397194194564...",40.023972,-75.356450,NaN,Pennsylvania,Philly art museum gazebo @noon,v-1595620736,NaN
6,[],False,4ca237fd5720b1f7046727ef,NaN,US,Oaks,United States,NaN,9079,"[Oaks, PA, United States]","[{'label': 'display', 'lat': 40.12361672753175...",40.123617,-75.454492,NaN,PA,American Treasure Tour,v-1595620736,35610900
7,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",False,4e764dda1495be517183427c,601 E Main St,US,Collegeville,United States,NaN,16355,"[601 E Main St, Collegeville, PA 19426, United...","[{'label': 'display', 'lat': 40.19394302368164...",40.193943,-75.455513,19426,PA,Berman Museum Of Art,v-1595620736,NaN
8,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",False,4c12c52b1b5cef3b5bd0edc4,1398 W Baltimore Pike,US,Media,United States,at Valley Rd.,17047,"[1398 W Baltimore Pike (at Valley Rd.), Media,...","[{'label': 'display', 'lat': 39.90467966872198...",39.904680,-75.452000,19063,PA,The Franklin Mint Museum,v-1595620736,NaN
9,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",False,4fb7cc2de4b00b64c4713dc6,301 W. Main Street,US,Trappe,United States,NaN,16439,"[301 W. Main Street, Trappe, PA 19426, United ...","[{'label': 'display', 'lat': 40.1973991394043,...",40.197399,-75.470970,19426,PA,Dewees museum,v-1595620736,NaN


In [78]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,American Helicopter Museum,Museum,1220 American Blvd,US,West Chester,United States,NaN,9180,"[1220 American Blvd, West Chester, PA 19380, U...","[{'label': 'display', 'lat': 39.99175853514661...",39.991759,-75.579209,19380,PA,4bc12eacb492d13a8cbda560
1,Wharton Esherick Museum,Museum,1520 Horse Shoe Trl,US,Malvern,United States,NaN,3690,"[1520 Horse Shoe Trl, Malvern, PA 19355, Unite...","[{'label': 'display', 'lat': 40.08381813883635...",40.083818,-75.493054,19355,PA,4e3d8227aeb73139a179eeb7
2,Train Museum Park,Historic Site,Newtown Square,US,Delaware County,United States,NaN,10634,"[Newtown Square, Delaware County, PA, United S...","[{'label': 'display', 'lat': 39.98138967391212...",39.981390,-75.422059,NaN,PA,4c797a2a566db60c1f21450e
3,Papermill House Museum,History Museum,NaN,US,Newtown Square,United States,NaN,9430,"[Newtown Square, PA, United States]","[{'label': 'display', 'lat': 40.01738209287441...",40.017382,-75.404970,NaN,PA,4e389b05fa763bfcc366590c
4,Newtown Railroad Museum,History Museum,4194-4222 W Chester Pike,US,Newtown Square,United States,Drexel Lodge Park,9700,"[4194-4222 W Chester Pike (Drexel Lodge Park),...","[{'label': 'display', 'lat': 39.974042, 'lng':...",39.974042,-75.455423,19073,PA,4da0aa1db3e7236a9613f878
5,Philly art museum gazebo @noon,None,NaN,US,NaN,United States,NaN,13115,"[Pennsylvania, United States]","[{'label': 'display', 'lat': 40.02397194194564...",40.023972,-75.356450,NaN,Pennsylvania,4f5ca513e4b0cfc5e1e4099e
6,American Treasure Tour,None,NaN,US,Oaks,United States,NaN,9079,"[Oaks, PA, United States]","[{'label': 'display', 'lat': 40.12361672753175...",40.123617,-75.454492,NaN,PA,4ca237fd5720b1f7046727ef
7,Berman Museum Of Art,Art Museum,601 E Main St,US,Collegeville,United States,NaN,16355,"[601 E Main St, Collegeville, PA 19426, United...","[{'label': 'display', 'lat': 40.19394302368164...",40.193943,-75.455513,19426,PA,4e764dda1495be517183427c
8,The Franklin Mint Museum,Museum,1398 W Baltimore Pike,US,Media,United States,at Valley Rd.,17047,"[1398 W Baltimore Pike (at Valley Rd.), Media,...","[{'label': 'display', 'lat': 39.90467966872198...",39.904680,-75.452000,19063,PA,4c12c52b1b5cef3b5bd0edc4
9,Dewees museum,History Museum,301 W. Main Street,US,Trappe,United States,NaN,16439,"[301 W. Main Street, Trappe, PA 19426, United ...","[{'label': 'display', 'lat': 40.1973991394043,...",40.197399,-75.470970,19426,PA,4fb7cc2de4b00b64c4713dc6


In [79]:
dataframe_filtered.shape

(12, 15)

In [80]:
df_museum_pa = dataframe_filtered

df_museum_pa.at[0:, 'categories']='Museum'
df_museum_pa

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,American Helicopter Museum,Museum,1220 American Blvd,US,West Chester,United States,NaN,9180,"[1220 American Blvd, West Chester, PA 19380, U...","[{'label': 'display', 'lat': 39.99175853514661...",39.991759,-75.579209,19380,PA,4bc12eacb492d13a8cbda560
1,Wharton Esherick Museum,Museum,1520 Horse Shoe Trl,US,Malvern,United States,NaN,3690,"[1520 Horse Shoe Trl, Malvern, PA 19355, Unite...","[{'label': 'display', 'lat': 40.08381813883635...",40.083818,-75.493054,19355,PA,4e3d8227aeb73139a179eeb7
2,Train Museum Park,Museum,Newtown Square,US,Delaware County,United States,NaN,10634,"[Newtown Square, Delaware County, PA, United S...","[{'label': 'display', 'lat': 39.98138967391212...",39.981390,-75.422059,NaN,PA,4c797a2a566db60c1f21450e
3,Papermill House Museum,Museum,NaN,US,Newtown Square,United States,NaN,9430,"[Newtown Square, PA, United States]","[{'label': 'display', 'lat': 40.01738209287441...",40.017382,-75.404970,NaN,PA,4e389b05fa763bfcc366590c
4,Newtown Railroad Museum,Museum,4194-4222 W Chester Pike,US,Newtown Square,United States,Drexel Lodge Park,9700,"[4194-4222 W Chester Pike (Drexel Lodge Park),...","[{'label': 'display', 'lat': 39.974042, 'lng':...",39.974042,-75.455423,19073,PA,4da0aa1db3e7236a9613f878
5,Philly art museum gazebo @noon,Museum,NaN,US,NaN,United States,NaN,13115,"[Pennsylvania, United States]","[{'label': 'display', 'lat': 40.02397194194564...",40.023972,-75.356450,NaN,Pennsylvania,4f5ca513e4b0cfc5e1e4099e
6,American Treasure Tour,Museum,NaN,US,Oaks,United States,NaN,9079,"[Oaks, PA, United States]","[{'label': 'display', 'lat': 40.12361672753175...",40.123617,-75.454492,NaN,PA,4ca237fd5720b1f7046727ef
7,Berman Museum Of Art,Museum,601 E Main St,US,Collegeville,United States,NaN,16355,"[601 E Main St, Collegeville, PA 19426, United...","[{'label': 'display', 'lat': 40.19394302368164...",40.193943,-75.455513,19426,PA,4e764dda1495be517183427c
8,The Franklin Mint Museum,Museum,1398 W Baltimore Pike,US,Media,United States,at Valley Rd.,17047,"[1398 W Baltimore Pike (at Valley Rd.), Media,...","[{'label': 'display', 'lat': 39.90467966872198...",39.904680,-75.452000,19063,PA,4c12c52b1b5cef3b5bd0edc4
9,Dewees museum,Museum,301 W. Main Street,US,Trappe,United States,NaN,16439,"[301 W. Main Street, Trappe, PA 19426, United ...","[{'label': 'display', 'lat': 40.1973991394043,...",40.197399,-75.470970,19426,PA,4fb7cc2de4b00b64c4713dc6


In [81]:
df_museum_pa.shape

(12, 15)

# Zoo

In [82]:
search_query = 'Zoo'
radius = 16093
print('done')

done


In [83]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Zoo&radius=16093&limit=500'

In [84]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3d209eba8045b92551a4'},
 'response': {'venues': [{'id': '4aabeed6f964a520095b20e3',
    'name': 'Elmwood Park Zoo',
    'location': {'address': '1661 Harding Boulevard',
     'lat': 40.13317819268151,
     'lng': -75.33762862040857,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.13317819268151,
       'lng': -75.33762862040857},
      {'label': 'entrance', 'lat': 40.13378, 'lng': -75.336119}],
     'distance': 16930,
     'postalCode': '19401',
     'cc': 'US',
     'city': 'Norristown',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['1661 Harding Boulevard',
      'Norristown, PA 19401',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d17b941735',
      'name': 'Zoo',
      'pluralName': 'Zoos',
      'shortName': 'Zoo',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/zoo_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'i

In [85]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d17b941735', 'name': 'Z...",False,4aabeed6f964a520095b20e3,1661 Harding Boulevard,US,Norristown,United States,16930,"[1661 Harding Boulevard, Norristown, PA 19401,...","[{'label': 'display', 'lat': 40.13317819268151...",40.133178,-75.337629,19401,PA,Elmwood Park Zoo,v-1595620736,46928930
1,"[{'id': '4bf58dd8d48988d1a0941735', 'name': 'C...",False,5048ccd1e4b09f6aaf120967,NaN,US,Broomall,United States,13472,"[Broomall, PA 19008, United States]","[{'label': 'display', 'lat': 39.96747194282841...",39.967472,-75.392978,19008,PA,Zoo-York,v-1595620736,NaN
2,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",False,4f789774e4b055c761c1eb78,NaN,US,Norristown,United States,16939,"[Norristown, PA 19401, United States]","[{'label': 'display', 'lat': 40.13337571335738...",40.133376,-75.337668,19401,PA,Elmwood Park Zoo Gift Shop,v-1595620736,NaN
3,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",False,5cdc308f9cadd9002b147136,915 S Trooper Rd,US,Norristown,United States,11318,"[915 S Trooper Rd, Norristown, PA 19403, Unite...","[{'label': 'display', 'lat': 40.1228004, 'lng'...",40.122800,-75.410299,19403,PA,Zoom Drain,v-1595620736,NaN
4,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",False,58366415d7b47337a08c5180,915 S Trooper Rd,US,Norristown,United States,11322,"[915 S Trooper Rd, Norristown, PA 19403, Unite...","[{'label': 'display', 'lat': 40.122822, 'lng':...",40.122822,-75.410265,19403,PA,Zoom Drain,v-1595620736,NaN
5,"[{'id': '4bf58dd8d48988d1a5941735', 'name': 'C...",False,503cc27ae4b0258ed7887fc4,NaN,US,Broomall,United States,13469,"[Broomall, PA 19008, United States]","[{'label': 'display', 'lat': 39.967571, 'lng':...",39.967571,-75.392903,19008,PA,Zoology Lab,v-1595620736,NaN
6,"[{'id': '4bf58dd8d48988d1a5941735', 'name': 'C...",False,5048ce42e4b09f6aaf12cbc2,NaN,US,Royersford,United States,15290,"[Royersford, PA 19468, United States]","[{'label': 'display', 'lat': 40.18856415720508...",40.188564,-75.526877,19468,PA,The Zoology Lab,v-1595620736,NaN
7,"[{'id': '4bf58dd8d48988d117941735', 'name': 'B...",False,5970f0a365211f46b1075b73,NaN,US,Norristown,United States,16870,"[Norristown, PA 19401, United States]","[{'label': 'display', 'lat': 40.132854, 'lng':...",40.132854,-75.338190,19401,PA,Zoobrew,v-1595620736,NaN
8,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'A...",False,5631d436498e0724878f597c,1256 E Ridge Pike,US,Plymouth Meeting,United States,16937,"[1256 E Ridge Pike, Plymouth Meeting, PA 19462...","[{'label': 'entrance', 'lat': 40.107795, 'lng'...",40.107632,-75.320709,19462,PA,CarZoom Auto Sales,v-1595620736,NaN
9,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",False,59a07bdb35d3fc19e5601135,1205 Pottstown Pike,US,Glenmoore,United States,17627,"[1205 Pottstown Pike, Glenmoore, PA 19343, Uni...","[{'label': 'display', 'lat': 40.11708831787109...",40.117088,-75.694641,19343,PA,Grandma Zook's Bakery & Produce,v-1595620736,NaN


In [86]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Elmwood Park Zoo,Zoo,1661 Harding Boulevard,US,Norristown,United States,16930,"[1661 Harding Boulevard, Norristown, PA 19401,...","[{'label': 'display', 'lat': 40.13317819268151...",40.133178,-75.337629,19401,PA,4aabeed6f964a520095b20e3
1,Zoo-York,College Classroom,NaN,US,Broomall,United States,13472,"[Broomall, PA 19008, United States]","[{'label': 'display', 'lat': 39.96747194282841...",39.967472,-75.392978,19008,PA,5048ccd1e4b09f6aaf120967
2,Elmwood Park Zoo Gift Shop,Gift Shop,NaN,US,Norristown,United States,16939,"[Norristown, PA 19401, United States]","[{'label': 'display', 'lat': 40.13337571335738...",40.133376,-75.337668,19401,PA,4f789774e4b055c761c1eb78
3,Zoom Drain,Construction & Landscaping,915 S Trooper Rd,US,Norristown,United States,11318,"[915 S Trooper Rd, Norristown, PA 19403, Unite...","[{'label': 'display', 'lat': 40.1228004, 'lng'...",40.122800,-75.410299,19403,PA,5cdc308f9cadd9002b147136
4,Zoom Drain,Construction & Landscaping,915 S Trooper Rd,US,Norristown,United States,11322,"[915 S Trooper Rd, Norristown, PA 19403, Unite...","[{'label': 'display', 'lat': 40.122822, 'lng':...",40.122822,-75.410265,19403,PA,58366415d7b47337a08c5180
5,Zoology Lab,College Lab,NaN,US,Broomall,United States,13469,"[Broomall, PA 19008, United States]","[{'label': 'display', 'lat': 39.967571, 'lng':...",39.967571,-75.392903,19008,PA,503cc27ae4b0258ed7887fc4
6,The Zoology Lab,College Lab,NaN,US,Royersford,United States,15290,"[Royersford, PA 19468, United States]","[{'label': 'display', 'lat': 40.18856415720508...",40.188564,-75.526877,19468,PA,5048ce42e4b09f6aaf12cbc2
7,Zoobrew,Beer Garden,NaN,US,Norristown,United States,16870,"[Norristown, PA 19401, United States]","[{'label': 'display', 'lat': 40.132854, 'lng':...",40.132854,-75.338190,19401,PA,5970f0a365211f46b1075b73
8,CarZoom Auto Sales,Auto Dealership,1256 E Ridge Pike,US,Plymouth Meeting,United States,16937,"[1256 E Ridge Pike, Plymouth Meeting, PA 19462...","[{'label': 'entrance', 'lat': 40.107795, 'lng'...",40.107632,-75.320709,19462,PA,5631d436498e0724878f597c
9,Grandma Zook's Bakery & Produce,Bakery,1205 Pottstown Pike,US,Glenmoore,United States,17627,"[1205 Pottstown Pike, Glenmoore, PA 19343, Uni...","[{'label': 'display', 'lat': 40.11708831787109...",40.117088,-75.694641,19343,PA,59a07bdb35d3fc19e5601135


In [87]:
dataframe_filtered.shape

(10, 14)

In [88]:
df_zoo_pa = dataframe_filtered[dataframe_filtered.categories == 'Zoo']
df_zoo_pa.reset_index(inplace=True, drop=True)
df_zoo_pa

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Elmwood Park Zoo,Zoo,1661 Harding Boulevard,US,Norristown,United States,16930,"[1661 Harding Boulevard, Norristown, PA 19401,...","[{'label': 'display', 'lat': 40.13317819268151...",40.133178,-75.337629,19401,PA,4aabeed6f964a520095b20e3


In [89]:
df_zoo_pa.shape

(1, 14)

# Golf Course

In [90]:
search_query = 'Golf Course'
radius = 16093
print('done')

done


In [91]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Golf Course&radius=16093&limit=500'

In [92]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3d018a8c2424434bae3f'},
 'response': {'venues': [{'id': '4ba5fd16f964a520002d39e3',
    'name': 'The Golf Course at Glen Mills',
    'location': {'address': '221 Glen Mills Rd',
     'lat': 39.917625510270796,
     'lng': -75.50745579999096,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.917625510270796,
       'lng': -75.50745579999096}],
     'distance': 14975,
     'postalCode': '19342',
     'cc': 'US',
     'city': 'Glen Mills',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['221 Glen Mills Rd',
      'Glen Mills, PA 19342',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e6941735',
      'name': 'Golf Course',
      'pluralName': 'Golf Courses',
      'shortName': 'Golf Course',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/golfcourse_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595620737',
    'hasPerk': Fals

In [93]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",False,4ba5fd16f964a520002d39e3,221 Glen Mills Rd,US,Glen Mills,United States,NaN,14975,"[221 Glen Mills Rd, Glen Mills, PA 19342, Unit...","[{'label': 'display', 'lat': 39.91762551027079...",39.917626,-75.507456,19342,PA,The Golf Course at Glen Mills,v-1595620737,NaN
1,"[{'id': '52e81612bcbc57f1066b79e8', 'name': 'D...",False,55835a27498e8832bd49aa2f,600 Lee Rd,US,Chesterbrook,United States,wilsondiscgolf.org,4647,"[600 Lee Rd (wilsondiscgolf.org), Chesterbrook...","[{'label': 'display', 'lat': 40.06821441650390...",40.068214,-75.455528,19087,PA,Wilson Farm Disc Golf Course,v-1595620737,NaN
2,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",False,4bdcd648c79cc928276088e9,401 S Schuylkill Ave #2,US,Eagleville,United States,NaN,12938,"[401 S Schuylkill Ave #2, Eagleville, PA 19403...","[{'label': 'display', 'lat': 40.12714004516601...",40.127140,-75.389801,19403,PA,Westover Golf Course,v-1595620737,NaN
3,"[{'id': '52e81612bcbc57f1066b79e8', 'name': 'D...",False,57964c06cd10b2eee5497012,Beidler Road,US,King of Prussia,United States,Henderson,12698,"[Beidler Road (Henderson), King of Prussia, PA...","[{'label': 'display', 'lat': 40.11195123489124...",40.111951,-75.378903,19406,PA,Sweetbriar Park Disc Golf Course,v-1595620737,NaN
4,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",False,4c190cf4fe5a76b0b1fa0215,2400 W Main St,US,Norristown,United States,NaN,14056,"[2400 W Main St, Norristown, PA 19403, United ...","[{'label': 'display', 'lat': 40.13858795166015...",40.138588,-75.385544,19403,PA,Jefferson Golf Course,v-1595620737,NaN
5,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",False,4da53fbba86e226c26b3ce67,66 Church Rd,US,Linfield,United States,NaN,18658,"[66 Church Rd, Linfield, PA 19468, United States]","[{'label': 'display', 'lat': 40.21079522410276...",40.210795,-75.576635,19468,PA,Linfield National Golf Course,v-1595620737,NaN
6,"[{'id': '52e81612bcbc57f1066b79e8', 'name': 'D...",False,567ea8cf498e4566a56fb962,1 Park Ln,US,Downingtown,United States,Wallace,17571,"[1 Park Ln (Wallace), Downingtown, PA 19335, U...","[{'label': 'display', 'lat': 40.006883, 'lng':...",40.006883,-75.703363,19335,PA,Kerr Park Disc Golf Course,v-1595620737,NaN
7,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",False,4c4b2f90959220a1327b380f,100 Line Rd,US,Malvern,United States,NaN,6105,"[100 Line Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 39.99996292269819...",39.999963,-75.527912,19355,PA,Applebrook Golf Club,v-1595620737,NaN
8,"[{'id': '4bf58dd8d48988d1f2941735', 'name': 'S...",False,5eb4596bb94fa90008e93e25,Swedesford Plaza,US,Berwyn,United States,NaN,5118,"[Swedesford Plaza, Berwyn, PA 19312, United St...","[{'label': 'display', 'lat': 40.0677197, 'lng'...",40.067720,-75.449349,19312,PA,Golf Galaxy,v-1595620737,NaN
9,"[{'id': '4bf58dd8d48988d1f2941735', 'name': 'S...",False,5ca7d538e47b4600397498e7,335 E King St,US,Malvern,United States,NaN,1651,"[335 E King St, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03767399999999...",40.037674,-75.510108,19355,PA,GolfTEC,v-1595620737,NaN


In [94]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Golf Course at Glen Mills,Golf Course,221 Glen Mills Rd,US,Glen Mills,United States,NaN,14975,"[221 Glen Mills Rd, Glen Mills, PA 19342, Unit...","[{'label': 'display', 'lat': 39.91762551027079...",39.917626,-75.507456,19342,PA,4ba5fd16f964a520002d39e3
1,Wilson Farm Disc Golf Course,Disc Golf,600 Lee Rd,US,Chesterbrook,United States,wilsondiscgolf.org,4647,"[600 Lee Rd (wilsondiscgolf.org), Chesterbrook...","[{'label': 'display', 'lat': 40.06821441650390...",40.068214,-75.455528,19087,PA,55835a27498e8832bd49aa2f
2,Westover Golf Course,Golf Course,401 S Schuylkill Ave #2,US,Eagleville,United States,NaN,12938,"[401 S Schuylkill Ave #2, Eagleville, PA 19403...","[{'label': 'display', 'lat': 40.12714004516601...",40.127140,-75.389801,19403,PA,4bdcd648c79cc928276088e9
3,Sweetbriar Park Disc Golf Course,Disc Golf,Beidler Road,US,King of Prussia,United States,Henderson,12698,"[Beidler Road (Henderson), King of Prussia, PA...","[{'label': 'display', 'lat': 40.11195123489124...",40.111951,-75.378903,19406,PA,57964c06cd10b2eee5497012
4,Jefferson Golf Course,Golf Course,2400 W Main St,US,Norristown,United States,NaN,14056,"[2400 W Main St, Norristown, PA 19403, United ...","[{'label': 'display', 'lat': 40.13858795166015...",40.138588,-75.385544,19403,PA,4c190cf4fe5a76b0b1fa0215
5,Linfield National Golf Course,Golf Course,66 Church Rd,US,Linfield,United States,NaN,18658,"[66 Church Rd, Linfield, PA 19468, United States]","[{'label': 'display', 'lat': 40.21079522410276...",40.210795,-75.576635,19468,PA,4da53fbba86e226c26b3ce67
6,Kerr Park Disc Golf Course,Disc Golf,1 Park Ln,US,Downingtown,United States,Wallace,17571,"[1 Park Ln (Wallace), Downingtown, PA 19335, U...","[{'label': 'display', 'lat': 40.006883, 'lng':...",40.006883,-75.703363,19335,PA,567ea8cf498e4566a56fb962
7,Applebrook Golf Club,Golf Course,100 Line Rd,US,Malvern,United States,NaN,6105,"[100 Line Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 39.99996292269819...",39.999963,-75.527912,19355,PA,4c4b2f90959220a1327b380f
8,Golf Galaxy,Sporting Goods Shop,Swedesford Plaza,US,Berwyn,United States,NaN,5118,"[Swedesford Plaza, Berwyn, PA 19312, United St...","[{'label': 'display', 'lat': 40.0677197, 'lng'...",40.067720,-75.449349,19312,PA,5eb4596bb94fa90008e93e25
9,GolfTEC,Sporting Goods Shop,335 E King St,US,Malvern,United States,NaN,1651,"[335 E King St, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03767399999999...",40.037674,-75.510108,19355,PA,5ca7d538e47b4600397498e7


In [95]:
dataframe_filtered.shape

(50, 15)

In [96]:
df_golf_pa = dataframe_filtered[dataframe_filtered.categories == 'Golf Course']
df_golf_pa.reset_index(inplace=True, drop=True)
df_golf_pa

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Golf Course at Glen Mills,Golf Course,221 Glen Mills Rd,US,Glen Mills,United States,NaN,14975,"[221 Glen Mills Rd, Glen Mills, PA 19342, Unit...","[{'label': 'display', 'lat': 39.91762551027079...",39.917626,-75.507456,19342,PA,4ba5fd16f964a520002d39e3
1,Westover Golf Course,Golf Course,401 S Schuylkill Ave #2,US,Eagleville,United States,NaN,12938,"[401 S Schuylkill Ave #2, Eagleville, PA 19403...","[{'label': 'display', 'lat': 40.12714004516601...",40.127140,-75.389801,19403,PA,4bdcd648c79cc928276088e9
2,Jefferson Golf Course,Golf Course,2400 W Main St,US,Norristown,United States,NaN,14056,"[2400 W Main St, Norristown, PA 19403, United ...","[{'label': 'display', 'lat': 40.13858795166015...",40.138588,-75.385544,19403,PA,4c190cf4fe5a76b0b1fa0215
3,Linfield National Golf Course,Golf Course,66 Church Rd,US,Linfield,United States,NaN,18658,"[66 Church Rd, Linfield, PA 19468, United States]","[{'label': 'display', 'lat': 40.21079522410276...",40.210795,-75.576635,19468,PA,4da53fbba86e226c26b3ce67
4,Applebrook Golf Club,Golf Course,100 Line Rd,US,Malvern,United States,NaN,6105,"[100 Line Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 39.99996292269819...",39.999963,-75.527912,19355,PA,4c4b2f90959220a1327b380f
5,Chester Valley Golf Club,Golf Course,430 Swedesford Rd,US,Malvern,United States,NaN,5029,"[430 Swedesford Rd, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.04214884119996...",40.042149,-75.563431,19355,PA,4bafec13f964a520bd2b3ce3
6,Spring Hollow Golf Club,Golf Course,3350 Schuylkill Rd,US,Spring City,United States,NaN,15957,"[3350 Schuylkill Rd, Spring City, PA 19475, Un...","[{'label': 'display', 'lat': 40.18550816675446...",40.185508,-75.574627,19475,PA,4bb295f035f0c9b60e83bb83
7,Grant Griffiths Golf,Golf Course,245 Lancaster Ave,US,Malvern,United States,NaN,3354,"[245 Lancaster Ave, Malvern, PA 19355, United ...","[{'label': 'display', 'lat': 40.0411962, 'lng'...",40.041196,-75.542540,19355,PA,5728ad35498ec8030de8104a
8,Aronimink Golf Club,Golf Course,3600 Saint Davids Rd,US,Newtown Square,United States,NaN,9393,"[3600 Saint Davids Rd, Newtown Square, PA 1907...","[{'label': 'display', 'lat': 40.0117074905324,...",40.011707,-75.409148,19073,PA,4be8496ac5220f47138da9ca
9,Jeffersonville Golf Club,Golf Course,2400 W Main St,US,Norristown,United States,btwn Trooper & Egypt Rd,13635,"[2400 W Main St (btwn Trooper & Egypt Rd), Nor...","[{'label': 'display', 'lat': 40.13665920533141...",40.136659,-75.389958,19403,PA,4bb8d7791261d13a1900e998


In [97]:
df_golf_pa.shape

(27, 15)

# Grade School

In [98]:
search_query = 'Grade School'
radius = 16093
print('done')

done


In [99]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Grade School&radius=16093&limit=500'

In [100]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3d4ed8d6542aaa6fb237'},
 'response': {'venues': [{'id': '4bf3089b98ac0f47d76b62a8',
    'name': 'Conestoga High School',
    'location': {'address': '200 Irish Rd',
     'lat': 40.04699873457121,
     'lng': -75.45306034415397,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.04699873457121,
       'lng': -75.45306034415397}],
     'distance': 4536,
     'postalCode': '19312',
     'cc': 'US',
     'city': 'Berwyn',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['200 Irish Rd',
      'Berwyn, PA 19312',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d13d941735',
      'name': 'High School',
      'pluralName': 'High Schools',
      'shortName': 'High School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595620737',
    'hasPerk': False},
   {'id': '4d238998dcf4224b31689c76

In [101]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4bf3089b98ac0f47d76b62a8,200 Irish Rd,US,Berwyn,United States,NaN,4536,"[200 Irish Rd, Berwyn, PA 19312, United States]","[{'label': 'display', 'lat': 40.04699873457121...",40.046999,-75.453060,19312,PA,Conestoga High School,v-1595620737,NaN
1,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",False,4d238998dcf4224b31689c76,583 Sugartown Rd,US,Malvern,United States,NaN,3292,"[583 Sugartown Rd, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.024321, 'lng':...",40.024321,-75.518948,19355,PA,The Phelps School,v-1595620737,NaN
2,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",False,5aca4dcf1acf111873fc6dc1,309 Lancaster Avenue,US,Malvern,United States,NaN,3879,"[309 Lancaster Avenue, Malvern, PA 19355, Unit...","[{'label': 'display', 'lat': 40.03989079999999...",40.039891,-75.548482,19355,PA,Goldfish Swim School - Malvern,v-1595620737,NaN
3,"[{'id': '52e81612bcbc57f1066b7a46', 'name': 'P...",False,4cfd70f4d7206ea8d5254969,418 S Warren Ave,US,Malvern,United States,NaN,2967,"[418 S Warren Ave, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.02590248814194...",40.025902,-75.511943,19355,PA,Malvern Preparatory School,v-1595620737,562665285
4,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",False,4f11f9bae4b0865b08348e4b,1 Liberty Blvd,US,Malvern,United States,NaN,1832,"[1 Liberty Blvd, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.05572, 'lng': ...",40.055720,-75.526862,19355,PA,Temple University Fox School of Business EMBA ...,v-1595620737,NaN
5,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",False,4be80601d837c9b65dc6a406,47 Church Rd,US,Malvern,United States,NaN,4612,"[47 Church Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.042342, 'lng':...",40.042342,-75.558459,19355,PA,Great valley school district office,v-1595620737,NaN
6,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",False,4b012d0bf964a520d24122e3,1777 N Valley Rd,US,Malvern,United States,at Yellow Springs Rd,2539,"[1777 N Valley Rd (at Yellow Springs Rd), Malv...","[{'label': 'display', 'lat': 40.07329464210996...",40.073295,-75.494682,19355,PA,The Vanguard School,v-1595620737,NaN
7,"[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,4ffdc880e4b06f1e866d1618,NaN,US,Malvern,United States,NaN,1299,"[Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.040986, 'lng':...",40.040986,-75.501397,19355,PA,International Montessori School,v-1595620737,NaN
8,"[{'id': '4f4532974b9074f6e4fb0104', 'name': 'D...",False,4d2db15588e237042a1c3cc7,250 Lancaster Avenue,US,Malvern,United States,NaN,3537,"[250 Lancaster Avenue, Malvern, PA 19355, Unit...","[{'label': 'display', 'lat': 40.0397142, 'lng'...",40.039714,-75.544069,19355,PA,The Goddard School,v-1595620737,NaN
9,"[{'id': '4f4533804b9074f6e4fb0105', 'name': 'E...",False,4c9bebd0542b224b1211dc9f,20 Devon Rd,US,Malvern,United States,NaN,2613,"[20 Devon Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03053425492323...",40.030534,-75.493902,19355,PA,General Wayne Elementary School,v-1595620737,NaN


In [102]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Conestoga High School,High School,200 Irish Rd,US,Berwyn,United States,NaN,4536,"[200 Irish Rd, Berwyn, PA 19312, United States]","[{'label': 'display', 'lat': 40.04699873457121...",40.046999,-75.453060,19312,PA,4bf3089b98ac0f47d76b62a8
1,The Phelps School,School,583 Sugartown Rd,US,Malvern,United States,NaN,3292,"[583 Sugartown Rd, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.024321, 'lng':...",40.024321,-75.518948,19355,PA,4d238998dcf4224b31689c76
2,Goldfish Swim School - Malvern,Athletics & Sports,309 Lancaster Avenue,US,Malvern,United States,NaN,3879,"[309 Lancaster Avenue, Malvern, PA 19355, Unit...","[{'label': 'display', 'lat': 40.03989079999999...",40.039891,-75.548482,19355,PA,5aca4dcf1acf111873fc6dc1
3,Malvern Preparatory School,Private School,418 S Warren Ave,US,Malvern,United States,NaN,2967,"[418 S Warren Ave, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.02590248814194...",40.025902,-75.511943,19355,PA,4cfd70f4d7206ea8d5254969
4,Temple University Fox School of Business EMBA ...,College Academic Building,1 Liberty Blvd,US,Malvern,United States,NaN,1832,"[1 Liberty Blvd, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.05572, 'lng': ...",40.055720,-75.526862,19355,PA,4f11f9bae4b0865b08348e4b
5,Great valley school district office,School,47 Church Rd,US,Malvern,United States,NaN,4612,"[47 Church Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.042342, 'lng':...",40.042342,-75.558459,19355,PA,4be80601d837c9b65dc6a406
6,The Vanguard School,School,1777 N Valley Rd,US,Malvern,United States,at Yellow Springs Rd,2539,"[1777 N Valley Rd (at Yellow Springs Rd), Malv...","[{'label': 'display', 'lat': 40.07329464210996...",40.073295,-75.494682,19355,PA,4b012d0bf964a520d24122e3
7,International Montessori School,Nursery School,NaN,US,Malvern,United States,NaN,1299,"[Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.040986, 'lng':...",40.040986,-75.501397,19355,PA,4ffdc880e4b06f1e866d1618
8,The Goddard School,Daycare,250 Lancaster Avenue,US,Malvern,United States,NaN,3537,"[250 Lancaster Avenue, Malvern, PA 19355, Unit...","[{'label': 'display', 'lat': 40.0397142, 'lng'...",40.039714,-75.544069,19355,PA,4d2db15588e237042a1c3cc7
9,General Wayne Elementary School,Elementary School,20 Devon Rd,US,Malvern,United States,NaN,2613,"[20 Devon Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03053425492323...",40.030534,-75.493902,19355,PA,4c9bebd0542b224b1211dc9f


In [103]:
dataframe_filtered.shape


(50, 15)

In [104]:

df1 = dataframe_filtered
df_school_pa = df1[(df1['categories']== 'School') | (df1['categories']=='Elementary School') | (df1['categories']== 'None') | (df1['categories'] =='Private School') | (df1['categories'] =='High School') | (df1['categories']== 'Middle School')]
df_school_pa.reset_index(inplace=True, drop=True)
df_school_pa.at[0:, 'categories']= 'School'
df_school_pa

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Conestoga High School,School,200 Irish Rd,US,Berwyn,United States,NaN,4536,"[200 Irish Rd, Berwyn, PA 19312, United States]","[{'label': 'display', 'lat': 40.04699873457121...",40.046999,-75.453060,19312,PA,4bf3089b98ac0f47d76b62a8
1,The Phelps School,School,583 Sugartown Rd,US,Malvern,United States,NaN,3292,"[583 Sugartown Rd, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.024321, 'lng':...",40.024321,-75.518948,19355,PA,4d238998dcf4224b31689c76
2,Malvern Preparatory School,School,418 S Warren Ave,US,Malvern,United States,NaN,2967,"[418 S Warren Ave, Malvern, PA 19355, United S...","[{'label': 'display', 'lat': 40.02590248814194...",40.025902,-75.511943,19355,PA,4cfd70f4d7206ea8d5254969
3,Great valley school district office,School,47 Church Rd,US,Malvern,United States,NaN,4612,"[47 Church Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.042342, 'lng':...",40.042342,-75.558459,19355,PA,4be80601d837c9b65dc6a406
4,The Vanguard School,School,1777 N Valley Rd,US,Malvern,United States,at Yellow Springs Rd,2539,"[1777 N Valley Rd (at Yellow Springs Rd), Malv...","[{'label': 'display', 'lat': 40.07329464210996...",40.073295,-75.494682,19355,PA,4b012d0bf964a520d24122e3
5,General Wayne Elementary School,School,20 Devon Rd,US,Malvern,United States,NaN,2613,"[20 Devon Rd, Malvern, PA 19355, United States]","[{'label': 'display', 'lat': 40.03053425492323...",40.030534,-75.493902,19355,PA,4c9bebd0542b224b1211dc9f
6,St Patrick's Elementary School,School,Changing Avenue,US,Malvern,United States,NaN,1929,"[Changing Avenue, Malvern, PA, United States]","[{'label': 'display', 'lat': 40.035421, 'lng':...",40.035421,-75.511803,NaN,PA,52db0864498e982a0782ceee
7,Hillside Elementary School,School,NaN,US,Berwyn,United States,NaN,4273,"[Berwyn, PA 19312, United States]","[{'label': 'display', 'lat': 40.056866, 'lng':...",40.056866,-75.456100,19312,PA,4bfe7bd4b68d0f47f13bea57
8,Royer-Greaves School for Blind,School,118 S Valley Rd,US,Paoli,United States,NaN,2605,"[118 S Valley Rd, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.036687, 'lng':...",40.036687,-75.482921,19301,PA,4ed81e9e8231efe38ca3390a
9,The Crossroads School,School,1681 N Valley Rd,US,Malvern,United States,Yellow Springs Rd,2711,"[1681 N Valley Rd (Yellow Springs Rd), Malvern...","[{'label': 'display', 'lat': 40.074303, 'lng':...",40.074303,-75.492651,19355,PA,4c459708da2176b08733637a


In [105]:
df_school_pa.shape

(29, 15)

# Library

In [106]:
search_query = 'Library'
radius = 16093
print('done')

done


In [107]:

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=Library&radius=16093&limit=500'

In [108]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b3d06b0d9d01cac5f802d'},
 'response': {'venues': [{'id': '4b3f6aa7f964a52091a625e3',
    'name': 'Tredyffrin Public Library',
    'location': {'address': '582 Upper Gulph Rd',
     'lat': 40.05678734342492,
     'lng': -75.4025140968484,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.05678734342492,
       'lng': -75.4025140968484}],
     'distance': 8821,
     'postalCode': '19087',
     'cc': 'US',
     'city': 'Strafford',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['582 Upper Gulph Rd',
      'Strafford, PA 19087',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d12f941735',
      'name': 'Library',
      'pluralName': 'Libraries',
      'shortName': 'Library',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595620738',
    'hasPerk': False},
   {'id': '4f5e35cce4b04f

In [109]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",False,4b3f6aa7f964a52091a625e3,582 Upper Gulph Rd,US,Strafford,United States,NaN,8821,"[582 Upper Gulph Rd, Strafford, PA 19087, Unit...","[{'label': 'display', 'lat': 40.05678734342492...",40.056787,-75.402514,19087,PA,Tredyffrin Public Library,v-1595620738,NaN
1,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,4f5e35cce4b04fe1105bf847,NaN,US,NaN,United States,NaN,1273,"[Pennsylvania, United States]","[{'label': 'display', 'lat': 40.04076703832841...",40.040767,-75.507393,NaN,Pennsylvania,Villa Library,v-1595620738,NaN
2,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,4d893e781508a143e43e0a1e,34 E. Swedesford Rd,US,Malvern,United States,NaN,1112,"[34 E. Swedesford Rd, Malvern, PA 19355, Unite...","[{'label': 'display', 'lat': 40.060082, 'lng':...",40.060082,-75.513813,19355,PA,PSGV library,v-1595620738,NaN
3,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",False,4b60d59bf964a52068fc29e3,1 E 1st Ave,US,Malvern,United States,Warren Ave.,2154,"[1 E 1st Ave (Warren Ave.), Malvern, PA 19355,...","[{'label': 'display', 'lat': 40.03339833759099...",40.033398,-75.512140,19355,PA,Malvern Public Library,v-1595620738,NaN
4,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",False,4b71e08bf964a520b7612de3,450 Exton Square Pkwy,US,Exton,United States,NaN,10061,"[450 Exton Square Pkwy, Exton, PA 19341, Unite...","[{'label': 'display', 'lat': 40.03181252392707...",40.031813,-75.620903,19341,PA,Chester County Library,v-1595620738,NaN
5,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,4b8aafd0f964a520be7932e3,25 W Rosedale Ave,US,West Chester,United States,at West Chester University,13594,[25 W Rosedale Ave (at West Chester University...,"[{'label': 'display', 'lat': 39.95204451829792...",39.952045,-75.597187,19383,PA,Francis Harvey Green Library,v-1595620738,NaN
6,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,4c8e33b9f1e8ef3bc9bd0bab,1145 King Rd,US,Malvern,United States,at Immaculata University,6053,"[1145 King Rd (at Immaculata University), Malv...","[{'label': 'display', 'lat': 40.02989246918784...",40.029892,-75.570679,19345,PA,Gabriele Library,v-1595620738,NaN
7,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,4b819295f964a520e3af30e3,18 Darby Rd,US,Paoli,United States,NaN,2260,"[18 Darby Rd, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04149500908692...",40.041495,-75.483293,19301,PA,Paoli Library,v-1595620738,NaN
8,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",False,4c1fed4bb306c928b33069b7,175 W Valley Forge Rd,US,King of Prussia,United States,NaN,12654,"[175 W Valley Forge Rd, King of Prussia, PA 19...","[{'label': 'display', 'lat': 40.10150973091726...",40.101510,-75.372039,19406,PA,Upper Merion Township Library,v-1595620738,NaN
9,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,4e6a397bd22d0e4cf5f1c156,1300 Eagle Rd,US,Wayne,United States,NaN,11563,"[1300 Eagle Rd, Wayne, PA 19087, United States]","[{'label': 'display', 'lat': 40.05052832799634...",40.050528,-75.370185,19087,PA,Warner Memorial Library,v-1595620738,NaN


In [110]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Tredyffrin Public Library,Library,582 Upper Gulph Rd,US,Strafford,United States,NaN,8821,"[582 Upper Gulph Rd, Strafford, PA 19087, Unit...","[{'label': 'display', 'lat': 40.05678734342492...",40.056787,-75.402514,19087,PA,4b3f6aa7f964a52091a625e3
1,Villa Library,College Library,NaN,US,NaN,United States,NaN,1273,"[Pennsylvania, United States]","[{'label': 'display', 'lat': 40.04076703832841...",40.040767,-75.507393,NaN,Pennsylvania,4f5e35cce4b04fe1105bf847
2,PSGV library,College Library,34 E. Swedesford Rd,US,Malvern,United States,NaN,1112,"[34 E. Swedesford Rd, Malvern, PA 19355, Unite...","[{'label': 'display', 'lat': 40.060082, 'lng':...",40.060082,-75.513813,19355,PA,4d893e781508a143e43e0a1e
3,Malvern Public Library,Library,1 E 1st Ave,US,Malvern,United States,Warren Ave.,2154,"[1 E 1st Ave (Warren Ave.), Malvern, PA 19355,...","[{'label': 'display', 'lat': 40.03339833759099...",40.033398,-75.512140,19355,PA,4b60d59bf964a52068fc29e3
4,Chester County Library,Library,450 Exton Square Pkwy,US,Exton,United States,NaN,10061,"[450 Exton Square Pkwy, Exton, PA 19341, Unite...","[{'label': 'display', 'lat': 40.03181252392707...",40.031813,-75.620903,19341,PA,4b71e08bf964a520b7612de3
5,Francis Harvey Green Library,College Library,25 W Rosedale Ave,US,West Chester,United States,at West Chester University,13594,[25 W Rosedale Ave (at West Chester University...,"[{'label': 'display', 'lat': 39.95204451829792...",39.952045,-75.597187,19383,PA,4b8aafd0f964a520be7932e3
6,Gabriele Library,College Library,1145 King Rd,US,Malvern,United States,at Immaculata University,6053,"[1145 King Rd (at Immaculata University), Malv...","[{'label': 'display', 'lat': 40.02989246918784...",40.029892,-75.570679,19345,PA,4c8e33b9f1e8ef3bc9bd0bab
7,Paoli Library,College Library,18 Darby Rd,US,Paoli,United States,NaN,2260,"[18 Darby Rd, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04149500908692...",40.041495,-75.483293,19301,PA,4b819295f964a520e3af30e3
8,Upper Merion Township Library,Library,175 W Valley Forge Rd,US,King of Prussia,United States,NaN,12654,"[175 W Valley Forge Rd, King of Prussia, PA 19...","[{'label': 'display', 'lat': 40.10150973091726...",40.101510,-75.372039,19406,PA,4c1fed4bb306c928b33069b7
9,Warner Memorial Library,College Library,1300 Eagle Rd,US,Wayne,United States,NaN,11563,"[1300 Eagle Rd, Wayne, PA 19087, United States]","[{'label': 'display', 'lat': 40.05052832799634...",40.050528,-75.370185,19087,PA,4e6a397bd22d0e4cf5f1c156


In [111]:

dataframe_filtered.shape

(50, 15)

In [112]:
df1 = dataframe_filtered
df2 = df1[df1.categories != 'Bar']
df3 = df2[df2.categories != 'College Bookstore']
df4 = df3[df3.categories != 'Trail']
df5 = df4[df4.categories != 'College Quad']
df6 = df5[df5.categories != 'Office']

df_library_pa = df6

df_library_pa.reset_index(inplace=True, drop=True)
df_library_pa.at[0:,'categories']= 'Library'
df_library_pa

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Tredyffrin Public Library,Library,582 Upper Gulph Rd,US,Strafford,United States,NaN,8821,"[582 Upper Gulph Rd, Strafford, PA 19087, Unit...","[{'label': 'display', 'lat': 40.05678734342492...",40.056787,-75.402514,19087,PA,4b3f6aa7f964a52091a625e3
1,Villa Library,Library,NaN,US,NaN,United States,NaN,1273,"[Pennsylvania, United States]","[{'label': 'display', 'lat': 40.04076703832841...",40.040767,-75.507393,NaN,Pennsylvania,4f5e35cce4b04fe1105bf847
2,PSGV library,Library,34 E. Swedesford Rd,US,Malvern,United States,NaN,1112,"[34 E. Swedesford Rd, Malvern, PA 19355, Unite...","[{'label': 'display', 'lat': 40.060082, 'lng':...",40.060082,-75.513813,19355,PA,4d893e781508a143e43e0a1e
3,Malvern Public Library,Library,1 E 1st Ave,US,Malvern,United States,Warren Ave.,2154,"[1 E 1st Ave (Warren Ave.), Malvern, PA 19355,...","[{'label': 'display', 'lat': 40.03339833759099...",40.033398,-75.512140,19355,PA,4b60d59bf964a52068fc29e3
4,Chester County Library,Library,450 Exton Square Pkwy,US,Exton,United States,NaN,10061,"[450 Exton Square Pkwy, Exton, PA 19341, Unite...","[{'label': 'display', 'lat': 40.03181252392707...",40.031813,-75.620903,19341,PA,4b71e08bf964a520b7612de3
5,Francis Harvey Green Library,Library,25 W Rosedale Ave,US,West Chester,United States,at West Chester University,13594,[25 W Rosedale Ave (at West Chester University...,"[{'label': 'display', 'lat': 39.95204451829792...",39.952045,-75.597187,19383,PA,4b8aafd0f964a520be7932e3
6,Gabriele Library,Library,1145 King Rd,US,Malvern,United States,at Immaculata University,6053,"[1145 King Rd (at Immaculata University), Malv...","[{'label': 'display', 'lat': 40.02989246918784...",40.029892,-75.570679,19345,PA,4c8e33b9f1e8ef3bc9bd0bab
7,Paoli Library,Library,18 Darby Rd,US,Paoli,United States,NaN,2260,"[18 Darby Rd, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04149500908692...",40.041495,-75.483293,19301,PA,4b819295f964a520e3af30e3
8,Upper Merion Township Library,Library,175 W Valley Forge Rd,US,King of Prussia,United States,NaN,12654,"[175 W Valley Forge Rd, King of Prussia, PA 19...","[{'label': 'display', 'lat': 40.10150973091726...",40.101510,-75.372039,19406,PA,4c1fed4bb306c928b33069b7
9,Warner Memorial Library,Library,1300 Eagle Rd,US,Wayne,United States,NaN,11563,"[1300 Eagle Rd, Wayne, PA 19087, United States]","[{'label': 'display', 'lat': 40.05052832799634...",40.050528,-75.370185,19087,PA,4e6a397bd22d0e4cf5f1c156


In [113]:
df_library_pa.shape

(45, 15)

# Combine

In [114]:
penn = pd.concat([df_church_pa, df_zoo_pa], sort=False)
penn = penn.append(df_aldi_pa)
penn = penn.append(df_gas_pa)
penn = penn.append(df_park_pa)
penn = penn.append(df_mall_pa)
penn = penn.append(df_thai_pa)
penn = penn.append(df_museum_pa)
penn = penn.append(df_golf_pa)
penn = penn.append(df_school_pa)
penn = penn.append(df_library_pa)
penn.reset_index(inplace=True, drop=True)
penn

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Church of Saint Monica,Church,63 Main Ave,US,Berwyn,United States,at First Ave,5623,"[63 Main Ave (at First Ave), Berwyn, PA 19312,...","[{'label': 'display', 'lat': 40.04608705725427...",40.046087,-75.440359,19312,PA,4e2ef003e4cdb5a70d631cec
1,Church of the Good Samaritan,Church,Lancaster Ave,US,Paoli,United States,NaN,1807,"[Lancaster Ave, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04091559619378...",40.040916,-75.490552,19301,PA,4b7c9e6ef964a520189e2fe3
2,St. Isaac Jogues Catholic Church,Church,50 W Walker Rd,US,Wayne,United States,NaN,6026,"[50 W Walker Rd, Wayne, PA 19087, United States]","[{'label': 'display', 'lat': 40.07388665956120...",40.073887,-75.441086,19087,PA,4ceb25c1baa6a1cde4fc3c6c
3,St Mary Of The Assumption Church,Church,199 St. Marys St.,US,Phoenixville,United States,Dayton St.,9852,"[199 St. Marys St. (Dayton St.), Phoenixville,...","[{'label': 'display', 'lat': 40.14046, 'lng': ...",40.140460,-75.513549,19460,PA,4d15785d6d103704171d2abd
4,"First Church Of Christ, Scientist",Church,NaN,US,Phoenixville,United States,NaN,1459,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.03923316394582...",40.039233,-75.508792,NaN,PA,539e0497498e71ce2a63cea3
5,Christian Science Church,Church,NaN,US,Phoenixville,United States,NaN,7620,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.12007149849076...",40.120071,-75.517039,NaN,PA,4e19b6f088777e96477c526e
6,The Methodist Church,Church,Landowner Road,US,Malvern,United States,NaN,1571,"[Landowner Road, Malvern, PA 19355, United Sta...","[{'label': 'display', 'lat': 40.038544, 'lng':...",40.038544,-75.510767,19355,PA,4ef6209e0aafbcb31b6551b9
7,St. David's Episcopal Church,Church,762 Valley Forge Rd,US,Wayne,United States,btwn Church Rd & Maplewood Rd,9127,[762 Valley Forge Rd (btwn Church Rd & Maplewo...,"[{'label': 'display', 'lat': 40.02697026100816...",40.026970,-75.403951,19087,PA,4e19d50dd4c062b044f53f58
8,"Saint Luke Lutheran Church - Devon, PA",Church,203 N Valley Forge Rd,US,Devon,United States,Conestoga Rd.,7212,"[203 N Valley Forge Rd (Conestoga Rd.), Devon,...","[{'label': 'display', 'lat': 40.05066761991265...",40.050668,-75.421251,19333,PA,4ec0362b30f82a2e13521d83
9,Great Valley Presbyterian Church,Church,2025 Swedesford Rd,US,Malvern,United States,NaN,1380,"[2025 Swedesford Rd, Malvern, PA 19355, United...","[{'label': 'display', 'lat': 40.06264683752855...",40.062647,-75.497251,19355,PA,4b9d4366f964a520319e36e3


In [115]:
penn.shape

(254, 15)

# Drop unneeded columns

In [116]:

penn.drop(['address', 'cc', 'country', 'crossStreet', 'distance', 'formattedAddress', 'labeledLatLngs', 'postalCode', 'state', 'id'], axis=1, inplace=True)
penn

,name,categories,city,lat,lng
0,The Church of Saint Monica,Church,Berwyn,40.046087,-75.440359
1,Church of the Good Samaritan,Church,Paoli,40.040916,-75.490552
2,St. Isaac Jogues Catholic Church,Church,Wayne,40.073887,-75.441086
3,St Mary Of The Assumption Church,Church,Phoenixville,40.140460,-75.513549
4,"First Church Of Christ, Scientist",Church,Phoenixville,40.039233,-75.508792
5,Christian Science Church,Church,Phoenixville,40.120071,-75.517039
6,The Methodist Church,Church,Malvern,40.038544,-75.510767
7,St. David's Episcopal Church,Church,Wayne,40.026970,-75.403951
8,"Saint Luke Lutheran Church - Devon, PA",Church,Devon,40.050668,-75.421251
9,Great Valley Presbyterian Church,Church,Malvern,40.062647,-75.497251


In [117]:
penn.shape

(254, 5)

# Rename Columns

In [118]:
penn = penn.rename(columns={'name': 'Name', 'categories': 'Categories', 'city': 'Neighborhood', 'lat': 'Latitude', 'lng': 'Longitude'})
penn

,Name,Categories,Neighborhood,Latitude,Longitude
0,The Church of Saint Monica,Church,Berwyn,40.046087,-75.440359
1,Church of the Good Samaritan,Church,Paoli,40.040916,-75.490552
2,St. Isaac Jogues Catholic Church,Church,Wayne,40.073887,-75.441086
3,St Mary Of The Assumption Church,Church,Phoenixville,40.140460,-75.513549
4,"First Church Of Christ, Scientist",Church,Phoenixville,40.039233,-75.508792
5,Christian Science Church,Church,Phoenixville,40.120071,-75.517039
6,The Methodist Church,Church,Malvern,40.038544,-75.510767
7,St. David's Episcopal Church,Church,Wayne,40.026970,-75.403951
8,"Saint Luke Lutheran Church - Devon, PA",Church,Devon,40.050668,-75.421251
9,Great Valley Presbyterian Church,Church,Malvern,40.062647,-75.497251


# Map

In [119]:
address = 'Malvern, PA'

geolocator = Nominatim(user_agent= "pa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of 100 Vanguard Blvd, Malvern, PA are {}, {}.'.format(latitude,longitude))

The geographical coordinate of 100 Vanguard Blvd, Malvern, PA are 31.1393439, -85.5191005.


In [120]:
map_pa = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(penn['Latitude'], penn['Longitude'], penn['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pa)

map_pa

In [121]:
penn.describe(include='all')

,Name,Categories,Neighborhood,Latitude,Longitude
count,254,254,243,254.000000,254.000000
unique,243,11,40,NaN,NaN
top,BP,Library,Malvern,NaN,NaN
freq,5,45,44,NaN,NaN
mean,NaN,NaN,NaN,40.051331,-75.480864
std,NaN,NaN,NaN,0.064068,0.083913
min,NaN,NaN,NaN,39.885068,-75.698385
25%,NaN,NaN,NaN,40.019670,-75.535039
50%,NaN,NaN,NaN,40.044153,-75.488587
75%,NaN,NaN,NaN,40.099680,-75.406741


In [122]:
penn = penn.sort_values(by=['Neighborhood'])
penn.reset_index(inplace=True, drop=True)
penn[200:]

,Name,Categories,Neighborhood,Latitude,Longitude
200,Reynolds Dog Park,Park,Upper Providence,40.129658,-75.482686
201,St. Thomas of Villanova Church,Church,Villanova,40.035814,-75.343358
202,Overbrook Golf Club,Golf Course,Villanova,40.011163,-75.372716
203,BP,Gas Station,Wayne,40.043426,-75.380509
204,St. Isaac Jogues Catholic Church,Church,Wayne,40.073887,-75.441086
205,Radnor Memorial Library,Library,Wayne,40.043044,-75.388851
206,Warner Memorial Library,Library,Wayne,40.050528,-75.370185
207,Church of the Saviour,Church,Wayne,40.053593,-75.390735
208,St. Matthews United Methodist Church of Valley...,Church,Wayne,40.081396,-75.421877
209,Devon Village Shopping Center,Shopping Mall,Wayne,40.048801,-75.411232


In [123]:
Audubon = penn[penn['Neighborhood'] == 'Audubon']
print(Audubon)
Audubon.groupby('Categories').count()
print('There are {} unique categories.'.format(len(Audubon['Categories'].unique())))

          Name   Categories Neighborhood  Latitude  Longitude
0  Liberty Gas  Gas Station      Audubon  40.11407 -75.419159
There are 1 unique categories.


In [124]:
Berwyn = penn[penn['Neighborhood'] == 'Berwyn']
print(Berwyn)
Berwyn.groupby('Categories').count()
print('There are {} unique categories.'.format(len(Berwyn['Categories'].unique())))

                                 Name       Categories Neighborhood  \
1          The Church of Saint Monica           Church       Berwyn   
2                      Teegarden Park             Park       Berwyn   
3         Valley Fair shopping Center    Shopping Mall       Berwyn   
4             Thai  Orchid Restaurant  Thai Restaurant       Berwyn   
5               Conestoga High School           School       Berwyn   
6          The Church Of Saint Monica           Church       Berwyn   
7   Tredyffrin-Easttown Middle School           School       Berwyn   
8                         CHS Library          Library       Berwyn   
9                    Easttown Library          Library       Berwyn   
10         Hillside Elementary School           School       Berwyn   

     Latitude  Longitude  
1   40.046087 -75.440359  
2   40.061483 -75.449172  
3   40.070389 -75.442851  
4   40.047695 -75.437849  
5   40.046999 -75.453060  
6   40.046040 -75.440033  
7   40.047807 -75.448091  
8 

In [125]:
Broomall = penn[penn['Neighborhood'] == 'Broomall']
print(Broomall)
Broomall.groupby('Categories').count()
print('There are {} unique categories.'.format(len(Broomall['Categories'].unique())))

                               Name     Categories Neighborhood   Latitude  \
11  St. Lukes Greek Orthodox Church         Church     Broomall  39.986178   
12       Mid-County Shopping Center  Shopping Mall     Broomall  39.976643   
13      Delaware County CC- Library        Library     Broomall  39.966818   
14     Broomall Presbyterian Church         Church     Broomall  39.985228   

    Longitude  
11 -75.366363  
12 -75.343799  
13 -75.392357  
14 -75.358078  
There are 3 unique categories.


In [126]:
Bryn_Mawr = penn[penn['Neighborhood'] == 'Bryn Mawr']
print(Bryn_Mawr)
Bryn_Mawr.groupby('Categories').count()
print('There are {} unique categories.'.format(len(Bryn_Mawr['Categories'].unique())))

                    Name   Categories Neighborhood   Latitude  Longitude
15     Ludington Library      Library    Bryn Mawr  40.020839 -75.317632
16               Golftec  Golf Course    Bryn Mawr  40.025918 -75.325295
17  Lower Merion Library      Library    Bryn Mawr  40.017859 -75.339171
There are 2 unique categories.


In [127]:
Chester = penn[penn['Neighborhood'] == 'Chester']
print(Chester)
Chester.groupby('Categories').count()
print('There are {} unique categories.'.format(len(Chester['Categories'].unique())))

                     Name   Categories Neighborhood  Latitude  Longitude
18  Meadowbrook Golf Club  Golf Course      Chester  40.11613 -75.522437
There are 1 unique categories.


In [128]:
x = 'Chester Springs'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                            Name   Categories     Neighborhood   Latitude  \
19       Chester Springs Library      Library  Chester Springs  40.100070   
20     Halman's Gulf Gas Station  Gas Station  Chester Springs  40.100359   
21  St. Matthews Lutheran Church       Church  Chester Springs  40.108218   

    Longitude  
19 -75.622539  
20 -75.612220  
21 -75.673388  
There are 3 unique categories.


In [129]:
x = 'Chesterbrook'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                    Name     Categories  Neighborhood  \
22  Chesterbrook Village Shopping Center  Shopping Mall  Chesterbrook   
23                      Wilson Farm Park           Park  Chesterbrook   

     Latitude  Longitude  
22  40.074061 -75.456950  
23  40.070877 -75.454338  
There are 2 unique categories.


In [130]:
x = 'Cheyney'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                            Name Categories Neighborhood   Latitude  Longitude
24  Leslie Pinckney Hill Library    Library      Cheyney  39.934353  -75.52844
There are 1 unique categories.


In [131]:
x = 'Collegeville'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                              Name       Categories  Neighborhood   Latitude  \
25            Berman Museum Of Art           Museum  Collegeville  40.193943   
26  Upper Providence Township Park             Park  Collegeville  40.162123   
27    Arrowhead  Elementary School           School  Collegeville  40.163489   
28               Thai Basil Bistro  Thai Restaurant  Collegeville  40.180203   

    Longitude  
25 -75.455513  
26 -75.488419  
27 -75.452721  
28 -75.454481  
There are 4 unique categories.


In [132]:
x = 'Delaware County'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                 Name Categories     Neighborhood  Latitude  Longitude
30  Train Museum Park     Museum  Delaware County  39.98139 -75.422059
There are 1 unique categories.


In [133]:
x = 'Devon'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                      Name     Categories Neighborhood  \
31                                    Aldi  Grocery Store        Devon   
32                                      BP    Gas Station        Devon   
33  Saint Luke Lutheran Church - Devon, PA         Church        Devon   
34            Devon Square Shopping Center  Shopping Mall        Devon   
35                             Liberty Gas    Gas Station        Devon   

     Latitude  Longitude  
31  40.047454 -75.413013  
32  40.048625 -75.414320  
33  40.050668 -75.421251  
34  40.045860 -75.408257  
35  40.046301 -75.422723  
There are 4 unique categories.


In [134]:
x = 'Downingtown'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                      Name Categories Neighborhood   Latitude  Longitude
36  Chester County Library    Library  Downingtown  40.008328 -75.698385
There are 1 unique categories.


In [135]:
x = 'Eagleville'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                  Name   Categories Neighborhood   Latitude  \
37  Lower Providence Community Library      Library   Eagleville  40.162112   
38                      East Coast Gas  Gas Station   Eagleville  40.132857   
39                Westover Golf Course  Golf Course   Eagleville  40.127140   

    Longitude  
37 -75.406708  
38 -75.378887  
39 -75.389801  
There are 3 unique categories.


In [136]:
x = 'East Whiteland Township'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

           Name   Categories             Neighborhood   Latitude  Longitude
40  Pioneer Gas  Gas Station  East Whiteland Township  40.035355 -75.570459
There are 1 unique categories.


In [137]:
x = 'Exton'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                     Name     Categories Neighborhood  \
41                               Speedway    Gas Station        Exton   
42                            Exton Plaza  Shopping Mall        Exton   
43                      Shell Gas Station    Gas Station        Exton   
44      Above Grade Level Delaware Valley         School        Exton   
45           Saints Philip & James School         School        Exton   
46                      Exton Square Mall  Shopping Mall        Exton   
47  Collegium Charter School 515 Building         School        Exton   
48                    West Whiteland Park           Park        Exton   
49                 Whiteland Towne Center  Shopping Mall        Exton   
50                 Chester County Library        Library        Exton   
51           Marsh Creek Community Church         Church        Exton   
52                                   ALDI  Grocery Store        Exton   
53              St Paul's Lutheran Church         C

In [138]:
x = 'Frazer'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                           Name     Categories Neighborhood   Latitude  \
54  Frazer View Shopping Center  Shopping Mall       Frazer  40.038738   

    Longitude  
54 -75.553735  
There are 1 unique categories.


In [139]:
x = 'Glen Mills'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                     Name   Categories Neighborhood  \
55  Chug Megill's Bar and Beer Can Museum       Museum   Glen Mills   
56          The Golf Course at Glen Mills  Golf Course   Glen Mills   
57                        Costco Gasoline  Gas Station   Glen Mills   

     Latitude  Longitude  
55  39.885068 -75.494824  
56  39.917626 -75.507456  
57  39.890485 -75.531198  
There are 3 unique categories.


In [140]:
x = 'King of Prussia'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                     Name       Categories     Neighborhood  \
58          Candlebrook Elementary School           School  King of Prussia   
59                   King of Prussia Mall    Shopping Mall  King of Prussia   
60           Valley Forge Shopping Center    Shopping Mall  King of Prussia   
61                            Heuser Park             Park  King of Prussia   
62                Caley Elementary School           School  King of Prussia   
63                                 Sunoco      Gas Station  King of Prussia   
64            Park Station - Port Kennedy             Park  King of Prussia   
65          Upper Merion Township Library          Library  King of Prussia   
66                       Gulf Gas Statuon      Gas Station  King of Prussia   
67                         Palmer Library          Library  King of Prussia   
68        Upper Merion Area Middle School           School  King of Prussia   
69                         Artillery Park           

In [141]:
x = 'Linfield'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                             Name   Categories Neighborhood   Latitude  \
74  Linfield National Golf Course  Golf Course     Linfield  40.210795   

    Longitude  
74 -75.576635  
There are 1 unique categories.


In [142]:
x = 'Lower Providence Township'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                        Name Categories               Neighborhood   Latitude  \
75  Lower Providence Library    Library  Lower Providence Township  40.162139   

    Longitude  
75 -75.406838  
There are 1 unique categories.


In [143]:
x = 'Malvern'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                         Name       Categories Neighborhood  \
76                    Malvern Shopping Center    Shopping Mall      Malvern   
77                 Malvern Preparatory School           School      Malvern   
78        Great valley school district office           School      Malvern   
79                               PSGV library          Library      Malvern   
80                     Malvern Public Library          Library      Malvern   
81                          The Phelps School           School      Malvern   
82        Theodore S. A. Rubino memorial Park             Park      Malvern   
83                        The Vanguard School           School      Malvern   
84               Great Valley Shopping Center    Shopping Mall      Malvern   
85                    Wharton Esherick Museum           Museum      Malvern   
86            General Wayne Elementary School           School      Malvern   
87             St Patrick's Elementary School       

In [144]:
x = 'Media'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                   Name Categories Neighborhood   Latitude  \
120            The Franklin Mint Museum     Museum        Media  39.904680   
121      Media-Upper Providence Library    Library        Media  39.918906   
122        Pennsylvania Veterans Museum     Museum        Media  39.917588   
123  Ridley Creek State Park Playground       Park        Media  39.947830   
124             Ridley Creek State Park       Park        Media  39.952813   

     Longitude  
120 -75.452000  
121 -75.388894  
122 -75.389106  
123 -75.449768  
124 -75.446334  
There are 3 unique categories.


In [145]:
x = 'Mont Clare'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

Empty DataFrame
Columns: [Name, Categories, Neighborhood, Latitude, Longitude]
Index: []
There are 0 unique categories.


In [146]:
x = 'Newtown Square'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))

                                 Name       Categories    Neighborhood  \
126               Liberty Gas Station      Gas Station  Newtown Square   
127           Newtown Railroad Museum           Museum  Newtown Square   
128            Papermill House Museum           Museum  Newtown Square   
129           Roger Annenburg Library          Library  Newtown Square   
130              St. Anastasia Church           Church  Newtown Square   
131             Sabai Thai Restaurant  Thai Restaurant  Newtown Square   
132               Aronimink Golf Club      Golf Course  Newtown Square   
133  Delaware County Christian School           School  Newtown Square   
134               Aronimink golf club      Golf Course  Newtown Square   
135                                BP      Gas Station  Newtown Square   
136                                BP      Gas Station  Newtown Square   
137            Olde Masters Golf Club      Golf Course  Newtown Square   
138                             Mobil 

In [147]:
x = 'Norristown'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                               Name     Categories Neighborhood   Latitude  \
140                Elmwood Park Zoo           Park   Norristown  40.133178   
141            Norristown Farm Park           Park   Norristown  40.144084   
142                Elmwood Park Zoo            Zoo   Norristown  40.133178   
143           Jefferson Golf Course    Golf Course   Norristown  40.138588   
144                Aldi Food Market  Grocery Store   Norristown  40.129382   
145             Liberty Gas Station    Gas Station   Norristown  40.124704   
146              Holy Savior Church         Church   Norristown  40.111720   
147               CITGO Gas Station    Gas Station   Norristown  40.115620   
148  Audubon Square Shopping Center  Shopping Mall   Norristown  40.128078   
149              Sunoco Gas Station    Gas Station   Norristown  40.108330   
150        Jeffersonville Golf Club    Golf Course   Norristown  40.136659   
151       Norristown Public Library        Library   Norristown 

'Norristown'

In [148]:
x = 'Oaks'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                       Name   Categories Neighborhood   Latitude  Longitude
152        BJ's Gas Station  Gas Station         Oaks  40.126870 -75.457098
153  American Treasure Tour       Museum         Oaks  40.123617 -75.454492
There are 2 unique categories.


'Oaks'

In [149]:
x = 'Paoli'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                               Name     Categories Neighborhood   Latitude  \
154    Church of the Good Samaritan         Church        Paoli  40.040916   
155            Paoli Baptist Church         Church        Paoli  40.037145   
156  Royer-Greaves School for Blind         School        Paoli  40.036687   
157                   Paoli Library        Library        Paoli  40.041495   
158  Delaware Valley Friends School         School        Paoli  40.044259   
159        Tredyffrin Township Park           Park        Paoli  40.043804   
160      Paoli West Shopping Center  Shopping Mall        Paoli  40.041660   
161           Paoli Shopping Center  Shopping Mall        Paoli  40.040840   
162                 Friendship Park           Park        Paoli  40.045339   
163                   Woodbine Park           Park        Paoli  40.045620   
164   Saint Norbert Catholic Church         Church        Paoli  40.037112   

     Longitude  
154 -75.490552  
155 -75.477924  
156 -75.4829

'Paoli'

In [150]:
x = 'Phoenixville'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                                  Name       Categories  Neighborhood  \
165           Christian Science Church           Church  Phoenixville   
166      Phoenixville Area High School           School  Phoenixville   
167                       PAHS Library          Library  Phoenixville   
168               Phoenixville Library          Library  Phoenixville   
169          Barkley Elementary School           School  Phoenixville   
170         The Malvern School of Oaks           School  Phoenixville   
171   St Mary Of The Assumption Church           Church  Phoenixville   
172  First Church Of Christ, Scientist           Church  Phoenixville   
173    Phoenixville Area Middle School           School  Phoenixville   
174                 Grace Bible Church           Church  Phoenixville   
175                Kimberton Golf Club      Golf Course  Phoenixville   
176                Reservior Dogs Park             Park  Phoenixville   
177               Lower Perkiomen Park             

'Phoenixville'

In [151]:
x = 'Radnor'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                         Name Categories Neighborhood   Latitude  Longitude
190       Holy Spirit Library    Library       Radnor  40.054980 -75.373688
191              Harford Park       Park       Radnor  40.052073 -75.355760
192  St Davids Community Park       Park       Radnor  40.043071 -75.375401
There are 2 unique categories.


'Radnor'

In [152]:
x = 'Rehoboth Beach'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                                       Name   Categories    Neighborhood  \
193  rookery north golf club Rehoboth Beach  Golf Course  Rehoboth Beach   

      Latitude  Longitude  
193  40.044193 -75.406485  
There are 1 unique categories.


'Rehoboth Beach'

In [153]:
x = 'Royersford'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                               Name Categories Neighborhood  Latitude  \
194  Royersford Free Public Library    Library   Royersford  40.18385   

     Longitude  
194 -75.536866  
There are 1 unique categories.


'Royersford'

In [154]:
x = 'Spring City'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                           Name   Categories Neighborhood   Latitude  \
195  Spring City Public Library      Library  Spring City  40.177603   
196   Turkey Hill Minit Markets  Gas Station  Spring City  40.181134   
197     Spring Hollow Golf Club  Golf Course  Spring City  40.185508   

     Longitude  
195 -75.549461  
196 -75.548183  
197 -75.574627  
There are 3 unique categories.


'Spring City'

In [155]:
x = 'Strafford'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                          Name Categories Neighborhood   Latitude  Longitude
198  Tredyffrin Public Library    Library    Strafford  40.056787 -75.402514
There are 1 unique categories.


'Strafford'

In [156]:
x = 'Trappe'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

              Name Categories Neighborhood   Latitude  Longitude
199  Dewees museum     Museum       Trappe  40.197399  -75.47097
There are 1 unique categories.


'Trappe'

In [157]:
x = 'Upper Providence'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                  Name Categories      Neighborhood   Latitude  Longitude
200  Reynolds Dog Park       Park  Upper Providence  40.129658 -75.482686
There are 1 unique categories.


'Upper Providence'

In [158]:
x = 'Villanova'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                               Name   Categories Neighborhood   Latitude  \
201  St. Thomas of Villanova Church       Church    Villanova  40.035814   
202             Overbrook Golf Club  Golf Course    Villanova  40.011163   

     Longitude  
201 -75.343358  
202 -75.372716  
There are 2 unique categories.


'Villanova'

In [159]:
x = 'Wayne'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                                                  Name     Categories  \
203                                                 BP    Gas Station   
204                   St. Isaac Jogues Catholic Church         Church   
205                            Radnor Memorial Library        Library   
206                            Warner Memorial Library        Library   
207                              Church of the Saviour         Church   
208  St. Matthews United Methodist Church of Valley...         Church   
209                      Devon Village Shopping Center  Shopping Mall   
210                                  Devon Prep School         School   
211                              St. David's Golf Club    Golf Course   
212                                Mangy Fox Golf Club    Golf Course   
213                            Gateway Shopping Center  Shopping Mall   
214                       St. David's Episcopal Church         Church   

    Neighborhood   Latitude  Longitude  
203      

'Wayne'

In [160]:
x = 'West Chester'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

                                                  Name     Categories  \
215                                Penn Oaks Golf Club    Golf Course   
216                   West Chester Little Free Library        Library   
217                                        Old Library        Library   
218                          First Presbyterian Church         Church   
219                                      Hendy Library        Library   
220                       Saints Peter And Paul Church         Church   
221            The Quiet Room (1st floor -FGH Library)        Library   
222                           Hershey's Mill Golf Club    Golf Course   
223                         Phillips Autograph Library        Library   
224                         American Helicopter Museum         Museum   
225                 Mt. Carmel Church Of God In Christ         Church   
226                        West Chester Public Library        Library   
227                        Rolling Green Memorial P

'West Chester'

In [161]:
x = 'West Norriton'

y = penn[penn['Neighborhood'] == x]
print(y)
y.groupby('Categories').count()
print('There are {} unique categories.'.format(len(y['Categories'].unique())))
x 

               Name   Categories   Neighborhood   Latitude  Longitude
242  76 Gas Station  Gas Station  West Norriton  39.950604 -75.363304
There are 1 unique categories.


'West Norriton'